# Dependencies for main video gui app

In [ ]:
!pip install opencv-python tk customtkinter pillow mediapipe numpy

# Main GUI app
* clean up but I don't know if this is the final version

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import tkinter as tk
from PIL import Image, ImageTk
import customtkinter as ck
import datetime
import os

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

current_exercise = None
recording = False
video_writer = None
video_counter = {}
draw_pose = False  # Variable to track if pose connections should be drawn


# Function to calculate angles between joints
def calculate_angle(a, b, c):
    a = np.array(a)  # First
    b = np.array(b)  # Mid
    c = np.array(c)  # End

    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(
        a[1] - b[1], a[0] - b[0]
    )
    angle = np.abs(radians * 180.0 / np.pi)

    if angle > 180.0:
        angle = 360 - angle

    return int(angle)


# Function to set up the webcam video capture
def video_setup(capture_device_index=0, frame_width=1280, frame_height=720):
    capture_device = cv2.VideoCapture(capture_device_index)
    capture_device.set(3, frame_width)
    capture_device.set(4, frame_height)
    return capture_device


# Function to process the frame with MediaPipe and overlay results
def process_frame(image, pose, exercise_type=None):
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image_rgb.flags.writeable = False
    results = pose.process(image_rgb)
    image_rgb.flags.writeable = True
    image_bgr = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR)

    if results.pose_landmarks:
        landmarks = results.pose_landmarks.landmark

        if exercise_type == "deadlift":
            shoulder = [
                landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y,
            ]
            hip = [
                landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y,
            ]
            knee = [
                landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y,
            ]
            angle = calculate_angle(shoulder, hip, knee)
            cv2.putText(
                image_bgr,
                str(angle),
                tuple(np.multiply(hip, [1280, 720]).astype(int)),
                cv2.FONT_HERSHEY_SIMPLEX,
                2,
                (255, 255, 255),
                2,
                cv2.LINE_AA,
            )

        # Additional configurations for squat and bench press can be added here

    if draw_pose:  # Draw pose connections only if draw_pose is True
        mp_drawing.draw_landmarks(
            image_bgr, results.pose_landmarks, mp_pose.POSE_CONNECTIONS
        )

    return image_bgr


# Function to start the live feed in the Tkinter window
def start_live_feed(capture, window, canvas, pose):
    global recording, video_writer
    ret, frame = capture.read()
    if ret:
        if current_exercise:
            frame = process_frame(frame, pose, current_exercise)
        else:
            frame = process_frame(frame, pose)

        if recording and video_writer:
            video_writer.write(frame)

        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGBA)
        captured_image = Image.fromarray(image)
        photo_image = ImageTk.PhotoImage(image=captured_image)

        canvas.create_image(0, 0, anchor=tk.NW, image=photo_image)
        canvas.photo_image = photo_image  # Keep a reference to avoid garbage collection

        window.after(10, start_live_feed, capture, window, canvas, pose)


# Function to capture the initial click position and calculate offset
def start_move_window(event, window):
    window._offset_x = event.x
    window._offset_y = event.y


# Function to move the window
def move_window(event, window):
    x = event.x_root - window._offset_x
    y = event.y_root - window._offset_y
    window.geometry(f"+{x}+{y}")


# Function to close the window
def close_window(window):
    window.destroy()


# Function to setup Tkinter window and buttons
def tk_setup(capture, pose, 
            dark_mode_bg_color = "#333333",       # Slightly lighter dark gray for the background
            dark_mode_title_color = "#181818",    # Darker gray for the title bar
            text_color = "#b0b0b0",               # Light Gray
            button_bg_color = "#4682b4",          # Steel Blue
            button_hover_color = "#315f72",       # Darker Steel Blue
            draw_button_color = "#40e0d0",        # Turquoise
            draw_button_hover_color = "#2e8b87",  # Darker Turquoise
            button_start_color = "#32cd32",       # Lime Green
            button_start_hover_color = "#228b22", # Darker Lime Green
            button_stop_color = "#ff6347",        # Tomato
            button_stop_hover_color = "#cd4f39",  # Darker Tomato
):
    window = tk.Tk()
    window.geometry("1280x800")
    window.title("Exercise Analyzer")
    window.configure(bg=dark_mode_bg_color)
    ck.set_appearance_mode("dark")
    window.overrideredirect(True)

    # Create a frame for the custom title bar
    title_bar = tk.Frame(window, bg=dark_mode_title_color, relief='raised', bd=1)
    title_bar.pack(side='top', fill='x')

    # Create a title label
    title_label = tk.Label(title_bar, text="Custom Title Bar", bg=dark_mode_title_color, fg='white')
    title_label.pack(side='left', padx=10)

    # Create a close button
    close_button = tk.Button(title_bar, text='x', command=lambda: close_window(window), bg=dark_mode_title_color, fg='white', bd=0)
    close_button.pack(side='right')

    # Bind the title bar to the move window function
    title_bar.bind('<Button-1>', lambda event: start_move_window(event, window))
    title_bar.bind('<B1-Motion>', lambda event: move_window(event,window))

    canvas = tk.Canvas(window, width=1280, height=600,highlightthickness=0,bd=0)
    canvas.pack()

    start_live_feed(capture, window, canvas, pose)

    # Frame to hold the buttons
    button_frame = tk.Frame(window)
    button_frame.configure(bg=dark_mode_bg_color)
    button_frame.pack(pady=20)  # Adjust the vertical space between canvas and buttons

    # First row of buttons
    deadlift_button = ck.CTkButton(
        button_frame,
        text="Deadlift",
        command=lambda: setup_exercise("deadlift"),
        height=40,
        width=120,
        font=("Arial", 20),
        text_color=text_color, 
        fg_color=button_bg_color,  
        hover_color=button_hover_color,  
    )
    deadlift_button.grid(row=0, column=0, padx=10, pady=10)

    squat_button = ck.CTkButton(
        button_frame,
        text="Squat",
        command=lambda: setup_exercise("squat"),
        height=40,
        width=120,
        font=("Arial", 20),
        text_color=text_color,
        fg_color=button_bg_color,
        hover_color=button_hover_color,
    )
    squat_button.grid(row=0, column=1, padx=10, pady=10)

    bench_button = ck.CTkButton(
        button_frame,
        text="Bench Press",
        command=lambda: setup_exercise("bench_press"),
        height=40,
        width=120,
        font=("Arial", 20),
        text_color=text_color,
        fg_color=button_bg_color,
        hover_color=button_hover_color,
    )
    bench_button.grid(row=0, column=2, padx=10, pady=10)

    # Second row of buttons
    draw_button = ck.CTkButton(
        button_frame,
        text="Draw",
        command=toggle_draw_pose,
        height=40,
        width=120,
        font=("Arial", 20),
        text_color=text_color,
        fg_color=draw_button_color,
        hover_color=draw_button_hover_color,
    )
    draw_button.grid(
        row=1, column=0, padx=10, pady=20
    )  # Increased pady for more vertical space

    record_button = ck.CTkButton(
        button_frame,
        text="Start Recording",
        command=start_recording,
        height=40,
        width=150,
        font=("Arial", 20),
        text_color=text_color,
        fg_color=button_start_color,
        hover_color=button_start_hover_color,
    )
    record_button.grid(
        row=1, column=1, padx=10, pady=20
    )  # Increased pady for more vertical space

    stop_button = ck.CTkButton(
        button_frame,
        text="Stop Recording",
        command=stop_recording,
        height=40,
        width=120,
        font=("Arial", 20),
        text_color=text_color,
        fg_color=button_stop_color,
        hover_color=button_stop_hover_color,
    )
    stop_button.grid(
        row=1, column=2, padx=10, pady=20
    )  # Increased pady for more vertical space

    return window


# Function to set the current exercise type
def setup_exercise(exercise):
    global current_exercise
    current_exercise = exercise


# Function to toggle drawing of pose connections
def toggle_draw_pose():
    global draw_pose
    draw_pose = not draw_pose


# Function to generate a unique filename for the recording
def generate_filename(exercise):
    global video_counter
    if exercise not in video_counter:
        video_counter[exercise] = 0
    video_counter[exercise] += 1
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    directory = f"{exercise}_video"
    if not os.path.exists(directory):
        os.makedirs(directory)
    return f"{directory}/{exercise}_{timestamp}_{video_counter[exercise]}.avi"


# Function to start recording
def start_recording():
    global recording, video_writer
    if current_exercise:
        filename = generate_filename(current_exercise)
        recording = True
        video_writer = cv2.VideoWriter(
            filename, cv2.VideoWriter_fourcc(*"MJPG"), 10, (1280, 720)
        )
        print(f"Recording started: {filename}")


# Function to stop recording
def stop_recording():
    global recording, video_writer
    recording = False
    if video_writer:
        video_writer.release()
        video_writer = None
        print("Recording stopped")


# Main function to initialize everything
def main():
    dark_mode_title_color = "#212121"
    dark_mode_bg_color = "#2f2f2f"
    button_text_color = "#282828"  # dim grey
    button_bg_color = "#1e90ff"  # electric blue
    button_hover_color = "#104e8b"  # darker
    draw_button_color = "#00ffff" # cyan
    draw_button_hover_color = "#008b8b"  # darker
    button_start_color = "#39ff14"  # neon green
    button_start_hover_color = "#228b22"  # darker
    button_stop_color = "#ff0000" # red
    button_stop_hover_color = "#8b0000"  # darker

    capture = video_setup()
    with mp_pose.Pose(
        min_detection_confidence=0.5, min_tracking_confidence=0.5
    ) as pose:
        tk_window = tk_setup(
            capture,
            pose,
            dark_mode_bg_color,
            dark_mode_title_color,
            button_text_color,
            button_bg_color,
            button_hover_color,
            draw_button_color,
            draw_button_hover_color,
            button_start_color,
            button_start_hover_color,
            button_stop_color,
            button_stop_hover_color,
        )
        tk_window.mainloop()

    capture.release()
    cv2.destroyAllWindows()


if __name__ == "__main__":
    main()

# Added Pandas

In [ ]:
pip install pandas

# Didn't use pandas here...hmm what was I doing

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
import tkinter as tk
from PIL import Image, ImageTk
import customtkinter as ck
import datetime
import os

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

data = []
current_exercise = None
recording = False
video_writer = None
video_counter = {}
draw_pose = False  # Variable to track if pose connections should be drawn


# Function to calculate angles between joints
def calculate_angle(a, b, c):
    a = np.array(a)  # First point
    b = np.array(b)  # Middle point (vertex of the angle)
    c = np.array(c)  # Last point

    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(
        a[1] - b[1], a[0] - b[0]
    )
    angle = np.abs(radians * 180.0 / np.pi)

    if angle > 180.0:
        angle = 360 - angle

    return angle


# Function to determine if the lift is successful
def determine_label(angle, exercise):
    if exercise == "deadlift":
        # For a deadlift, the back angle should be between 160 and 180 degrees
        if angle >= 160 and angle <= 180:
            return 1  # 1 indicates success
        else:
            return 0  # 0 indicates failure

    elif exercise == "squat":
        # For a squat, the knee angle should be less than or equal to 90 degrees
        if angle <= 90:
            return 1  # 1 indicates success
        else:
            return 0  # 0 indicates failure

    elif exercise == "bench_press":
        # For a bench press, the elbow angle should be between 70 and 110 degrees
        if angle >= 70 and angle <= 110:
            return 1  # 1 indicates success
        else:
            return 0  # 0 indicates failure

    return 0  # Default to failure if criteria not met


# Functions to analyze angles for each exercise
def analyze_deadlift(landmarks):
    shoulder = [
        landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
        landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y,
    ]
    hip = [
        landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
        landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y,
    ]
    knee = [
        landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
        landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y,
    ]
    back_angle = calculate_angle(shoulder, hip, knee)
    label = determine_label(back_angle, "deadlift")
    return back_angle, label


def analyze_squat(landmarks):
    hip = [
        landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
        landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y,
    ]
    knee = [
        landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
        landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y,
    ]
    ankle = [
        landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
        landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y,
    ]
    knee_angle = calculate_angle(hip, knee, ankle)
    label = determine_label(knee_angle, "squat")
    return knee_angle, label


def analyze_bench_press(landmarks):
    shoulder = [
        landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
        landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y,
    ]
    elbow = [
        landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
        landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y,
    ]
    wrist = [
        landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
        landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y,
    ]
    elbow_angle = calculate_angle(shoulder, elbow, wrist)
    label = determine_label(elbow_angle, "bench_press")
    return elbow_angle, label


# Function to process the frame with MediaPipe and overlay results
def process_frame(image, pose, exercise_type=None, collect_data=False):
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image_rgb.flags.writeable = False
    results = pose.process(image_rgb)
    image_rgb.flags.writeable = True
    image_bgr = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR)

    if results.pose_landmarks:
        landmarks = results.pose_landmarks.landmark

        if exercise_type == "deadlift":
            back_angle, label = analyze_deadlift(landmarks)
            cv2.putText(
                image_bgr,
                f"Back Angle: {back_angle:.2f} ({label})",
                (50, 50),
                cv2.FONT_HERSHEY_SIMPLEX,
                1,
                (255, 255, 255),
                2,
                cv2.LINE_AA,
            )
            if collect_data:
                save_features(landmarks, exercise_type)

        elif exercise_type == "squat":
            knee_angle, label = analyze_squat(landmarks)
            cv2.putText(
                image_bgr,
                f"Knee Angle: {knee_angle:.2f} ({label})",
                (50, 50),
                cv2.FONT_HERSHEY_SIMPLEX,
                1,
                (255, 255, 255),
                2,
                cv2.LINE_AA,
            )
            if collect_data:
                save_features(landmarks, exercise_type)

        elif exercise_type == "bench_press":
            elbow_angle, label = analyze_bench_press(landmarks)
            cv2.putText(
                image_bgr,
                f"Elbow Angle: {elbow_angle:.2f} ({label})",
                (50, 50),
                cv2.FONT_HERSHEY_SIMPLEX,
                1,
                (255, 255, 255),
                2,
                cv2.LINE_AA,
            )
            if collect_data:
                save_features(landmarks, exercise_type)

        if draw_pose:
            mp_drawing.draw_landmarks(
                image_bgr, results.pose_landmarks, mp_pose.POSE_CONNECTIONS
            )

    return image_bgr


# Function to save features to the data list
def save_features(landmarks, exercise):
    if exercise == "deadlift":
        back_angle, label = analyze_deadlift(landmarks)
        data.append([back_angle, label])

    elif exercise == "squat":
        knee_angle, label = analyze_squat(landmarks)
        data.append([knee_angle, label])

    elif exercise == "bench_press":
        elbow_angle, label = analyze_bench_press(landmarks)
        data.append([elbow_angle, label])


# Function to set up the webcam video capture
def video_setup(capture_device_index=0, frame_width=1280, frame_height=720):
    capture_device = cv2.VideoCapture(capture_device_index)
    capture_device.set(3, frame_width)
    capture_device.set(4, frame_height)
    return capture_device


# Function to start the live feed in the Tkinter window
def start_live_feed(capture, window, canvas, pose):
    global recording, video_writer
    ret, frame = capture.read()
    if ret:
        if current_exercise:
            frame = process_frame(frame, pose, current_exercise)
        else:
            frame = process_frame(frame, pose)

        if recording and video_writer:
            video_writer.write(frame)

        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGBA)
        captured_image = Image.fromarray(image)
        photo_image = ImageTk.PhotoImage(image=captured_image)

        canvas.create_image(0, 0, anchor=tk.NW, image=photo_image)
        canvas.photo_image = photo_image  # Keep a reference to avoid garbage collection

        window.after(10, start_live_feed, capture, window, canvas, pose)


# Function to capture the initial click position and calculate offset
def start_move_window(event, window):
    window._offset_x = event.x
    window._offset_y = event.y


# Function to move the window
def move_window(event, window):
    x = event.x_root - window._offset_x
    y = event.y_root - window._offset_y
    window.geometry(f"+{x}+{y}")


# Function to close the window
def close_window(window):
    window.destroy()


# Function to setup Tkinter window and buttons
def tk_setup(
    capture,
    pose,
    dark_mode_bg_color="#333333",  # Slightly lighter dark gray for the background
    dark_mode_title_color="#181818",  # Darker gray for the title bar
    text_color="#b0b0b0",  # Light Gray
    button_bg_color="#4682b4",  # Steel Blue
    button_hover_color="#315f72",  # Darker Steel Blue
    draw_button_color="#40e0d0",  # Turquoise
    draw_button_hover_color="#2e8b87",  # Darker Turquoise
    button_start_color="#32cd32",  # Lime Green
    button_start_hover_color="#228b22",  # Darker Lime Green
    button_stop_color="#ff6347",  # Tomato
    button_stop_hover_color="#cd4f39",  # Darker Tomato
):
    window = tk.Tk()
    window.geometry("1280x800")
    window.title("Exercise Analyzer")
    window.configure(bg=dark_mode_bg_color)
    ck.set_appearance_mode("dark")
    window.overrideredirect(True)

    # Create a frame for the custom title bar
    title_bar = tk.Frame(window, bg=dark_mode_title_color, relief="raised", bd=1)
    title_bar.pack(side="top", fill="x")

    # Create a title label
    title_label = tk.Label(
        title_bar, text="Custom Title Bar", bg=dark_mode_title_color, fg="white"
    )
    title_label.pack(side="left", padx=10)

    # Create a close button
    close_button = tk.Button(
        title_bar,
        text="x",
        command=lambda: close_window(window),
        bg=dark_mode_title_color,
        fg="white",
        bd=0,
    )
    close_button.pack(side="right")

    # Bind the title bar to the move window function
    title_bar.bind("<Button-1>", lambda event: start_move_window(event, window))
    title_bar.bind("<B1-Motion>", lambda event: move_window(event, window))

    canvas = tk.Canvas(window, width=1280, height=600, highlightthickness=0, bd=0)
    canvas.pack()

    start_live_feed(capture, window, canvas, pose)

    # Frame to hold the buttons
    button_frame = tk.Frame(window)
    button_frame.configure(bg=dark_mode_bg_color)
    button_frame.pack(pady=20)  # Adjust the vertical space between canvas and buttons

    # First row of buttons
    deadlift_button = ck.CTkButton(
        button_frame,
        text="Deadlift",
        command=lambda: setup_exercise("deadlift"),
        height=40,
        width=120,
        font=("Arial", 20),
        text_color=text_color,
        fg_color=button_bg_color,
        hover_color=button_hover_color,
    )
    deadlift_button.grid(row=0, column=0, padx=10, pady=10)

    squat_button = ck.CTkButton(
        button_frame,
        text="Squat",
        command=lambda: setup_exercise("squat"),
        height=40,
        width=120,
        font=("Arial", 20),
        text_color=text_color,
        fg_color=button_bg_color,
        hover_color=button_hover_color,
    )
    squat_button.grid(row=0, column=1, padx=10, pady=10)

    bench_button = ck.CTkButton(
        button_frame,
        text="Bench Press",
        command=lambda: setup_exercise("bench_press"),
        height=40,
        width=120,
        font=("Arial", 20),
        text_color=text_color,
        fg_color=button_bg_color,
        hover_color=button_hover_color,
    )
    bench_button.grid(row=0, column=2, padx=10, pady=10)

    # Second row of buttons
    draw_button = ck.CTkButton(
        button_frame,
        text="Draw",
        command=toggle_draw_pose,
        height=40,
        width=120,
        font=("Arial", 20),
        text_color=text_color,
        fg_color=draw_button_color,
        hover_color=draw_button_hover_color,
    )
    draw_button.grid(
        row=1, column=0, padx=10, pady=20
    )  # Increased pady for more vertical space

    record_button = ck.CTkButton(
        button_frame,
        text="Start Recording",
        command=start_recording,
        height=40,
        width=150,
        font=("Arial", 20),
        text_color=text_color,
        fg_color=button_start_color,
        hover_color=button_start_hover_color,
    )
    record_button.grid(
        row=1, column=1, padx=10, pady=20
    )  # Increased pady for more vertical space

    stop_button = ck.CTkButton(
        button_frame,
        text="Stop Recording",
        command=stop_recording,
        height=40,
        width=120,
        font=("Arial", 20),
        text_color=text_color,
        fg_color=button_stop_color,
        hover_color=button_stop_hover_color,
    )
    stop_button.grid(
        row=1, column=2, padx=10, pady=20
    )  # Increased pady for more vertical space

    return window


# Function to set the current exercise type
def setup_exercise(exercise):
    global current_exercise
    current_exercise = exercise


# Function to toggle drawing of pose connections
def toggle_draw_pose():
    global draw_pose
    draw_pose = not draw_pose


# Function to generate a unique filename for the recording
def generate_filename(exercise):
    global video_counter
    if exercise not in video_counter:
        video_counter[exercise] = 0
    video_counter[exercise] += 1
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    directory = f"{exercise}_video"
    if not os.path.exists(directory):
        os.makedirs(directory)
    return f"{directory}/{exercise}_{timestamp}_{video_counter[exercise]}.avi"


# Function to start recording
def start_recording():
    global recording, video_writer
    if current_exercise:
        filename = generate_filename(current_exercise)
        recording = True
        video_writer = cv2.VideoWriter(
            filename, cv2.VideoWriter_fourcc(*"MJPG"), 10, (1280, 720)
        )
        print(f"Recording started: {filename}")


# Function to stop recording
def stop_recording():
    global recording, video_writer
    recording = False
    if video_writer:
        video_writer.release()
        video_writer = None
        print("Recording stopped")


# Main function to initialize everything
def main():
    dark_mode_title_color = "#212121"
    dark_mode_bg_color = "#2f2f2f"
    button_text_color = "#282828"  # dim grey
    button_bg_color = "#1e90ff"  # electric blue
    button_hover_color = "#104e8b"  # darker
    draw_button_color = "#00ffff"  # cyan
    draw_button_hover_color = "#008b8b"  # darker
    button_start_color = "#39ff14"  # neon green
    button_start_hover_color = "#228b22"  # darker
    button_stop_color = "#ff0000"  # red
    button_stop_hover_color = "#8b0000"  # darker

    capture = video_setup()
    with mp_pose.Pose(
        min_detection_confidence=0.5, min_tracking_confidence=0.5
    ) as pose:
        tk_window = tk_setup(
            capture,
            pose,
            dark_mode_bg_color,
            dark_mode_title_color,
            button_text_color,
            button_bg_color,
            button_hover_color,
            draw_button_color,
            draw_button_hover_color,
            button_start_color,
            button_start_hover_color,
            button_stop_color,
            button_stop_hover_color,
        )
        tk_window.mainloop()

    capture.release()
    cv2.destroyAllWindows()


if __name__ == "__main__":
    main()

# Or here. Did I actually use pandas in the main app?
* I was going to originally use it to put the landmarks in a csv for the later model training
    * but I think that I didn't do that

In [3]:
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
import tkinter as tk
from PIL import Image, ImageTk
import customtkinter as ck
import datetime
import os

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

data = []
current_exercise = None
recording = False
video_writer = None
video_counter = {}
draw_pose = False  # Variable to track if pose connections should be drawn
start_detected = False
end_detected = False
previous_knee_angle = None
previous_back_angle = None
failed_lift = False


# Function to calculate angles between joints
def calculate_angle(a, b, c):
    a = np.array(a)  # First point
    b = np.array(b)  # Middle point (vertex of the angle)
    c = np.array(c)  # Last point

    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(
        a[1] - b[1], a[0] - b[0]
    )
    angle = np.abs(radians * 180.0 / np.pi)

    if angle > 180.0:
        angle = 360 - angle

    return angle


# Function to determine if the lift is successful
def determine_label(angle, exercise):
    if exercise == "deadlift":
        # For a deadlift, the back angle should be between 160 and 180 degrees
        if angle >= 160 and angle <= 180:
            return 1  # 1 indicates success
        else:
            return 0  # 0 indicates failure

    elif exercise == "squat":
        # For a squat, the knee angle should be less than or equal to 90 degrees
        if angle <= 90:
            return 1  # 1 indicates success
        else:
            return 0  # 0 indicates failure

    elif exercise == "bench_press":
        # For a bench press, the elbow angle should be between 70 and 110 degrees
        if angle >= 70 and angle <= 110:
            return 1  # 1 indicates success
        else:
            return 0  # 0 indicates failure

    return 0  # Default to failure if criteria not met


# Functions to analyze angles for each exercise
def analyze_deadlift(landmarks, start_detected, end_detected):
    global previous_knee_angle, previous_back_angle, failed_lift
    shoulder = [
        landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
        landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y,
    ]
    hip = [
        landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
        landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y,
    ]
    knee = [
        landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
        landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y,
    ]
    ankle = [
        landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
        landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y,
    ]

    back_angle = calculate_angle(shoulder, hip, knee)
    knee_angle = calculate_angle(hip, knee, ankle)

    # Detect start position
    if knee_angle < 45 and back_angle < 45 and not start_detected:
        start_detected = True
        end_detected = False  # Reset end detection when starting a new lift
        failed_lift = False

    # Detect end position
    if knee_angle > 160 and back_angle > 160 and start_detected:
        end_detected = True

    # Check for failure conditions
    if start_detected and not end_detected:
        if previous_knee_angle is not None and previous_back_angle is not None:
            if (
                knee_angle < previous_knee_angle - 20
                or back_angle < previous_back_angle - 20
            ):
                failed_lift = True

    # Update previous angles
    previous_knee_angle = knee_angle
    previous_back_angle = back_angle

    # Determine label based on end position and failure detection
    label = 1 if start_detected and end_detected and not failed_lift else 0

    # Reset start and end if successfully hitting end point
    # Not necessarily a successful rep
    if end_detected:
        start_detected = False
        end_detected = False

    return back_angle, knee_angle, label, start_detected, end_detected


def analyze_squat(landmarks, start_detected, end_detected):
    hip = [
        landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
        landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y,
    ]
    knee = [
        landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
        landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y,
    ]
    ankle = [
        landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
        landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y,
    ]

    knee_angle = calculate_angle(hip, knee, ankle)
    hip_angle = calculate_angle(
        [
            landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
            landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y,
        ],
        hip,
        knee,
    )

    # Detect start position
    if knee_angle > 160 and hip_angle > 160 and not start_detected:
        start_detected = True
        end_detected = False  # Reset end detection when starting a new squat

    # Detect end position
    if knee_angle < 90 and hip_angle < 90 and start_detected:
        end_detected = True

    # Determine label based on end position detection
    label = 1 if start_detected and end_detected else 0

    # Reset start and end if successfully hitting end point
    # Not necessarily a successful rep
    if end_detected:
        start_detected = False
        end_detected = False

    return knee_angle, hip_angle, label, start_detected, end_detected


def analyze_bench_press(landmarks, start_detected, end_detected):
    shoulder = [
        landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
        landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y,
    ]
    elbow = [
        landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
        landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y,
    ]
    wrist = [
        landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
        landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y,
    ]

    elbow_angle = calculate_angle(shoulder, elbow, wrist)

    # Detect start position
    if elbow_angle > 160 and not start_detected:
        start_detected = True
        end_detected = False  # Reset end detection when starting a new bench press

    # Detect end position
    if elbow_angle < 70 and start_detected:
        end_detected = True

    # Determine label based on end position detection
    label = 1 if start_detected and end_detected else 0

    # Reset start and end if successfully hitting end point
    # Not necessarily a successful rep
    if end_detected:
        start_detected = False
        end_detected = False

    return elbow_angle, label, start_detected, end_detected


# Function to process the frame with MediaPipe and overlay results
def process_frame(image, pose, exercise_type=None, collect_data=False):
    global start_detected, end_detected, data
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image_rgb.flags.writeable = False
    results = pose.process(image_rgb)
    image_rgb.flags.writeable = True
    image_bgr = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR)

    if results.pose_landmarks:
        landmarks = results.pose_landmarks.landmark

        if exercise_type == "deadlift" and draw_pose:
            back_angle, knee_angle, label, start_detected, end_detected = (
                analyze_deadlift(landmarks, start_detected, end_detected)
            )
            cv2.putText(
                image_bgr,
                f"Back Angle: {back_angle:.2f}",
                (50, 50),
                cv2.FONT_HERSHEY_SIMPLEX,
                1,
                (255, 255, 255),
                2,
                cv2.LINE_AA,
            )
            cv2.putText(
                image_bgr,
                f"Knee Angle: {knee_angle:.2f}",
                (50, 100),
                cv2.FONT_HERSHEY_SIMPLEX,
                1,
                (255, 255, 255),
                2,
                cv2.LINE_AA,
            )
            cv2.putText(
                image_bgr,
                f"Label: {label}",
                (50, 150),
                cv2.FONT_HERSHEY_SIMPLEX,
                1,
                (255, 255, 255),
                2,
                cv2.LINE_AA,
            )
            cv2.putText(
                image_bgr,
                f"Position Start: {start_detected}",
                (600, 50),
                cv2.FONT_HERSHEY_SIMPLEX,
                1,
                (255, 255, 255),
                2,
                cv2.LINE_AA,
            )
            cv2.putText(
                image_bgr,
                f"Position End: {end_detected}",
                (600, 100),
                cv2.FONT_HERSHEY_SIMPLEX,
                1,
                (255, 255, 255),
                2,
                cv2.LINE_AA,
            )

            if collect_data:
                save_features(landmarks, exercise_type)

        elif exercise_type == "squat" and draw_pose:
            knee_angle, hip_angle, label, start_detected, end_detected = analyze_squat(
                landmarks, start_detected, end_detected
            )
            cv2.putText(
                image_bgr,
                f"Knee Angle: {knee_angle:.2f}",
                (50, 50),
                cv2.FONT_HERSHEY_SIMPLEX,
                1,
                (255, 255, 255),
                2,
                cv2.LINE_AA,
            )
            cv2.putText(
                image_bgr,
                f"Hip Angle: {hip_angle:.2f}",
                (50, 100),
                cv2.FONT_HERSHEY_SIMPLEX,
                1,
                (255, 255, 255),
                2,
                cv2.LINE_AA,
            )
            cv2.putText(
                image_bgr,
                f"Label: {label}",
                (50, 150),
                cv2.FONT_HERSHEY_SIMPLEX,
                1,
                (255, 255, 255),
                2,
                cv2.LINE_AA,
            )
            cv2.putText(
                image_bgr,
                f"Position Start: {start_detected}",
                (600, 50),
                cv2.FONT_HERSHEY_SIMPLEX,
                1,
                (255, 255, 255),
                2,
                cv2.LINE_AA,
            )
            cv2.putText(
                image_bgr,
                f"Position End: {end_detected}",
                (600, 100),
                cv2.FONT_HERSHEY_SIMPLEX,
                1,
                (255, 255, 255),
                2,
                cv2.LINE_AA,
            )
            if collect_data:
                save_features(landmarks, exercise_type)

        elif exercise_type == "bench_press" and draw_pose:
            elbow_angle, label, start_detected, end_detected = analyze_bench_press(
                landmarks, start_detected, end_detected
            )
            cv2.putText(
                image_bgr,
                f"Elbow Angle: {elbow_angle:.2f}",
                (50, 50),
                cv2.FONT_HERSHEY_SIMPLEX,
                1,
                (255, 255, 255),
                2,
                cv2.LINE_AA,
            )
            cv2.putText(
                image_bgr,
                f"Label: {label}",
                (50, 100),
                cv2.FONT_HERSHEY_SIMPLEX,
                1,
                (255, 255, 255),
                2,
                cv2.LINE_AA,
            )
            cv2.putText(
                image_bgr,
                f"Position Start: {start_detected}",
                (600, 50),
                cv2.FONT_HERSHEY_SIMPLEX,
                1,
                (255, 255, 255),
                2,
                cv2.LINE_AA,
            )
            cv2.putText(
                image_bgr,
                f"Position End: {end_detected}",
                (600, 100),
                cv2.FONT_HERSHEY_SIMPLEX,
                1,
                (255, 255, 255),
                2,
                cv2.LINE_AA,
            )
            if collect_data:
                save_features(landmarks, exercise_type)

        if draw_pose:
            mp_drawing.draw_landmarks(
                image_bgr, results.pose_landmarks, mp_pose.POSE_CONNECTIONS
            )

    return image_bgr


def save_features(landmarks, exercise):
    if exercise == "deadlift":
        back_angle, knee_angle, label, _, _ = analyze_deadlift(
            landmarks, start_detected, end_detected
        )
        data.append([back_angle, knee_angle, label])

    elif exercise == "squat":
        knee_angle, hip_angle, label, _, _ = analyze_squat(
            landmarks, start_detected, end_detected
        )
        data.append([knee_angle, hip_angle, label])

    elif exercise == "bench_press":
        elbow_angle, label, _, _ = analyze_bench_press(
            landmarks, start_detected, end_detected
        )
        data.append([elbow_angle, label])


# Function to set up the webcam video capture
def video_setup(capture_device_index=0, frame_width=1280, frame_height=720):
    capture_device = cv2.VideoCapture(capture_device_index)
    capture_device.set(3, frame_width)
    capture_device.set(4, frame_height)
    return capture_device


# Function to start the live feed in the Tkinter window
def start_live_feed(capture, window, canvas, pose):
    global recording, video_writer
    ret, frame = capture.read()
    if ret:
        if current_exercise:
            frame = process_frame(frame, pose, current_exercise)
        else:
            frame = process_frame(frame, pose)

        if recording and video_writer:
            video_writer.write(frame)

        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGBA)
        captured_image = Image.fromarray(image)
        photo_image = ImageTk.PhotoImage(image=captured_image)

        canvas.create_image(0, 0, anchor=tk.NW, image=photo_image)
        canvas.photo_image = photo_image  # Keep a reference to avoid garbage collection

        window.after(10, start_live_feed, capture, window, canvas, pose)


# Function to capture the initial click position and calculate offset
def start_move_window(event, window):
    window._offset_x = event.x
    window._offset_y = event.y


# Function to move the window
def move_window(event, window):
    x = event.x_root - window._offset_x
    y = event.y_root - window._offset_y
    window.geometry(f"+{x}+{y}")


# Function to close the window
def close_window(window):
    window.destroy()


# Function to setup Tkinter window and buttons
def tk_setup(
    capture,
    pose,
    dark_mode_bg_color="#333333",  # Slightly lighter dark gray for the background
    dark_mode_title_color="#181818",  # Darker gray for the title bar
    text_color="#b0b0b0",  # Light Gray
    button_bg_color="#4682b4",  # Steel Blue
    button_hover_color="#315f72",  # Darker Steel Blue
    draw_button_color="#40e0d0",  # Turquoise
    draw_button_hover_color="#2e8b87",  # Darker Turquoise
    button_start_color="#32cd32",  # Lime Green
    button_start_hover_color="#228b22",  # Darker Lime Green
    button_stop_color="#ff6347",  # Tomato
    button_stop_hover_color="#cd4f39",  # Darker Tomato
):
    window = tk.Tk()
    window.geometry("1280x800")
    window.title("Exercise Analyzer")
    window.configure(bg=dark_mode_bg_color)
    ck.set_appearance_mode("dark")
    window.overrideredirect(True)

    # Create a frame for the custom title bar
    title_bar = tk.Frame(window, bg=dark_mode_title_color, relief="raised", bd=1)
    title_bar.pack(side="top", fill="x")

    # Create a title label
    title_label = tk.Label(
        title_bar, text="Custom Title Bar", bg=dark_mode_title_color, fg="white"
    )
    title_label.pack(side="left", padx=10)

    # Create a close button
    close_button = tk.Button(
        title_bar,
        text="x",
        command=lambda: close_window(window),
        bg=dark_mode_title_color,
        fg="white",
        bd=0,
    )
    close_button.pack(side="right")

    # Bind the title bar to the move window function
    title_bar.bind("<Button-1>", lambda event: start_move_window(event, window))
    title_bar.bind("<B1-Motion>", lambda event: move_window(event, window))

    canvas = tk.Canvas(window, width=1280, height=600, highlightthickness=0, bd=0)
    canvas.pack()

    start_live_feed(capture, window, canvas, pose)

    # Frame to hold the buttons
    button_frame = tk.Frame(window)
    button_frame.configure(bg=dark_mode_bg_color)
    button_frame.pack(pady=20)  # Adjust the vertical space between canvas and buttons

    # First row of buttons
    deadlift_button = ck.CTkButton(
        button_frame,
        text="Deadlift",
        command=lambda: setup_exercise("deadlift"),
        height=40,
        width=120,
        font=("Arial", 20),
        text_color=text_color,
        fg_color=button_bg_color,
        hover_color=button_hover_color,
    )
    deadlift_button.grid(row=0, column=0, padx=10, pady=10)

    squat_button = ck.CTkButton(
        button_frame,
        text="Squat",
        command=lambda: setup_exercise("squat"),
        height=40,
        width=120,
        font=("Arial", 20),
        text_color=text_color,
        fg_color=button_bg_color,
        hover_color=button_hover_color,
    )
    squat_button.grid(row=0, column=1, padx=10, pady=10)

    bench_button = ck.CTkButton(
        button_frame,
        text="Bench Press",
        command=lambda: setup_exercise("bench_press"),
        height=40,
        width=120,
        font=("Arial", 20),
        text_color=text_color,
        fg_color=button_bg_color,
        hover_color=button_hover_color,
    )
    bench_button.grid(row=0, column=2, padx=10, pady=10)

    # Second row of buttons
    draw_button = ck.CTkButton(
        button_frame,
        text="Draw",
        command=toggle_draw_pose,
        height=40,
        width=120,
        font=("Arial", 20),
        text_color=text_color,
        fg_color=draw_button_color,
        hover_color=draw_button_hover_color,
    )
    draw_button.grid(
        row=1, column=0, padx=10, pady=20
    )  # Increased pady for more vertical space

    record_button = ck.CTkButton(
        button_frame,
        text="Start Recording",
        command=start_recording,
        height=40,
        width=150,
        font=("Arial", 20),
        text_color=text_color,
        fg_color=button_start_color,
        hover_color=button_start_hover_color,
    )
    record_button.grid(
        row=1, column=1, padx=10, pady=20
    )  # Increased pady for more vertical space

    stop_button = ck.CTkButton(
        button_frame,
        text="Stop Recording",
        command=stop_recording,
        height=40,
        width=120,
        font=("Arial", 20),
        text_color=text_color,
        fg_color=button_stop_color,
        hover_color=button_stop_hover_color,
    )
    stop_button.grid(
        row=1, column=2, padx=10, pady=20
    )  # Increased pady for more vertical space

    return window


# Function to set the current exercise type
def setup_exercise(exercise):
    global current_exercise
    current_exercise = exercise


# Function to toggle drawing of pose connections
def toggle_draw_pose():
    global draw_pose
    draw_pose = not draw_pose


# Function to generate a unique filename for the recording
def generate_filename(exercise):
    global video_counter
    if exercise not in video_counter:
        video_counter[exercise] = 0
    video_counter[exercise] += 1
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    directory = f"{exercise}_video"
    if not os.path.exists(directory):
        os.makedirs(directory)
    return f"{directory}/{exercise}_{timestamp}_{video_counter[exercise]}.avi"


# Function to start recording
def start_recording():
    global recording, video_writer
    if current_exercise:
        filename = generate_filename(current_exercise)
        recording = True
        video_writer = cv2.VideoWriter(
            filename, cv2.VideoWriter_fourcc(*"MJPG"), 10, (1280, 720)
        )
        print(f"Recording started: {filename}")


# Function to stop recording
def stop_recording():
    global recording, video_writer
    recording = False
    if video_writer:
        video_writer.release()
        video_writer = None
        print("Recording stopped")


# Main function to initialize everything
def main():
    dark_mode_title_color = "#212121"
    dark_mode_bg_color = "#2f2f2f"
    button_text_color = "#282828"  # dim grey
    button_bg_color = "#1e90ff"  # electric blue
    button_hover_color = "#104e8b"  # darker
    draw_button_color = "#00ffff"  # cyan
    draw_button_hover_color = "#008b8b"  # darker
    button_start_color = "#39ff14"  # neon green
    button_start_hover_color = "#228b22"  # darker
    button_stop_color = "#ff0000"  # red
    button_stop_hover_color = "#8b0000"  # darker

    capture = video_setup()
    with mp_pose.Pose(
        min_detection_confidence=0.5, min_tracking_confidence=0.5
    ) as pose:
        tk_window = tk_setup(
            capture,
            pose,
            dark_mode_bg_color,
            dark_mode_title_color,
            button_text_color,
            button_bg_color,
            button_hover_color,
            draw_button_color,
            draw_button_hover_color,
            button_start_color,
            button_start_hover_color,
            button_stop_color,
            button_stop_hover_color,
        )
        tk_window.mainloop()

    capture.release()
    cv2.destroyAllWindows()


if __name__ == "__main__":
    main()

Recording started: bench_press_video/bench_press_20240727_142712_1.avi
Recording stopped


# I think this is the newest GUI App
* it has 2 different save locations


In [2]:
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
import tkinter as tk
from PIL import Image, ImageTk
import customtkinter as ck
import datetime
import os

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

data = []
current_exercise = None
recording = False
video_writer = None
counter = {}
draw_pose = False  # Variable to track if pose connections should be drawn
start_detected = False
end_detected = False
previous_knee_angle = None
previous_back_angle = None
failed_lift = False
photos_dir = None


# Function to calculate angles between joints
def calculate_angle(a, b, c):
    a = np.array(a)  # First point
    b = np.array(b)  # Middle point (vertex of the angle)
    c = np.array(c)  # Last point

    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(
        a[1] - b[1], a[0] - b[0]
    )
    angle = np.abs(radians * 180.0 / np.pi)

    if angle > 180.0:
        angle = 360 - angle

    return angle


# Function to determine if the lift is successful
def determine_label(angle, exercise):
    if exercise == "deadlift":
        # For a deadlift, the back angle should be between 160 and 180 degrees
        if angle >= 160 and angle <= 180:
            return 1  # 1 indicates success
        else:
            return 0  # 0 indicates failure

    elif exercise == "squat":
        # For a squat, the knee angle should be less than or equal to 90 degrees
        if angle <= 90:
            return 1  # 1 indicates success
        else:
            return 0  # 0 indicates failure

    elif exercise == "bench_press":
        # For a bench press, the elbow angle should be between 70 and 110 degrees
        if angle >= 70 and angle <= 110:
            return 1  # 1 indicates success
        else:
            return 0  # 0 indicates failure

    return 0  # Default to failure if criteria not met


# Functions to analyze angles for each exercise
def analyze_deadlift(landmarks, start_detected, end_detected):
    global previous_knee_angle, previous_back_angle, failed_lift
    shoulder = [
        landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
        landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y,
    ]
    hip = [
        landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
        landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y,
    ]
    knee = [
        landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
        landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y,
    ]
    ankle = [
        landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
        landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y,
    ]

    back_angle = calculate_angle(shoulder, hip, knee)
    knee_angle = calculate_angle(hip, knee, ankle)

    # Detect start position
    if knee_angle < 45 and back_angle < 45 and not start_detected:
        start_detected = True
        end_detected = False  # Reset end detection when starting a new lift
        failed_lift = False

    # Detect end position
    if knee_angle > 160 and back_angle > 160 and start_detected:
        end_detected = True

    # Check for failure conditions
    if start_detected and not end_detected:
        if previous_knee_angle is not None and previous_back_angle is not None:
            if (
                knee_angle < previous_knee_angle - 20
                or back_angle < previous_back_angle - 20
            ):
                failed_lift = True

    # Update previous angles
    previous_knee_angle = knee_angle
    previous_back_angle = back_angle

    # Determine label based on end position and failure detection
    label = 1 if start_detected and end_detected and not failed_lift else 0

    # Reset start and end if successfully hitting end point
    # Not necessarily a successful rep
    if end_detected:
        start_detected = False
        end_detected = False

    return back_angle, knee_angle, label, start_detected, end_detected


def analyze_squat(landmarks, start_detected, end_detected):
    hip = [
        landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
        landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y,
    ]
    knee = [
        landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
        landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y,
    ]
    ankle = [
        landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
        landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y,
    ]
    shoulder = [
        landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
        landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y,
    ]
    knee_angle = calculate_angle(hip, knee, ankle)
    hip_angle = calculate_angle(shoulder, hip, knee)

    # Detect start position
    if knee_angle > 160 and hip_angle > 160 and not start_detected:
        start_detected = True
        end_detected = False  # Reset end detection when starting a new squat

    # Detect end position
    if knee_angle < 90 and hip_angle < 90 and start_detected:
        end_detected = True

    # Determine label based on end position detection
    label = 1 if start_detected and end_detected else 0

    # Reset start and end if successfully hitting end point
    # Not necessarily a successful rep
    if end_detected:
        start_detected = False
        end_detected = False

    return knee_angle, hip_angle, label, start_detected, end_detected


def analyze_bench_press(landmarks, start_detected, end_detected):
    shoulder = [
        landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
        landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y,
    ]
    elbow = [
        landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
        landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y,
    ]
    wrist = [
        landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
        landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y,
    ]

    elbow_angle = calculate_angle(shoulder, elbow, wrist)

    # Detect start position
    if elbow_angle > 160 and not start_detected:
        start_detected = True
        end_detected = False  # Reset end detection when starting a new bench press

    # Detect end position
    if elbow_angle < 70 and start_detected:
        end_detected = True

    # Determine label based on end position detection
    label = 1 if start_detected and end_detected else 0

    # Reset start and end if successfully hitting end point
    # Not necessarily a successful rep
    if end_detected:
        start_detected = False
        end_detected = False

    return elbow_angle, label, start_detected, end_detected


# Function to process the frame with MediaPipe and overlay results
def process_frame(image, pose, exercise_type=None, collect_data=False):
    global start_detected, end_detected, data
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image_rgb.flags.writeable = False
    results = pose.process(image_rgb)
    image_rgb.flags.writeable = True
    image_bgr = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR)

    if results.pose_landmarks:
        landmarks = results.pose_landmarks.landmark

        if exercise_type == "deadlift" and draw_pose:
            back_angle, knee_angle, label, start_detected, end_detected = (
                analyze_deadlift(landmarks, start_detected, end_detected)
            )
            cv2.putText(
                image_bgr,
                f"Back Angle: {back_angle:.2f}",
                (50, 50),
                cv2.FONT_HERSHEY_SIMPLEX,
                1,
                (255, 255, 255),
                2,
                cv2.LINE_AA,
            )
            cv2.putText(
                image_bgr,
                f"Knee Angle: {knee_angle:.2f}",
                (50, 100),
                cv2.FONT_HERSHEY_SIMPLEX,
                1,
                (255, 255, 255),
                2,
                cv2.LINE_AA,
            )
            cv2.putText(
                image_bgr,
                f"Label: {label}",
                (50, 150),
                cv2.FONT_HERSHEY_SIMPLEX,
                1,
                (255, 255, 255),
                2,
                cv2.LINE_AA,
            )
            if collect_data:
                save_features(landmarks, exercise_type)

        elif exercise_type == "squat" and draw_pose:
            knee_angle, hip_angle, label, start_detected, end_detected = analyze_squat(
                landmarks, start_detected, end_detected
            )
            cv2.putText(
                image_bgr,
                f"Knee Angle: {knee_angle:.2f}",
                (50, 50),
                cv2.FONT_HERSHEY_SIMPLEX,
                1,
                (255, 255, 255),
                2,
                cv2.LINE_AA,
            )
            cv2.putText(
                image_bgr,
                f"Hip Angle: {hip_angle:.2f}",
                (50, 100),
                cv2.FONT_HERSHEY_SIMPLEX,
                1,
                (255, 255, 255),
                2,
                cv2.LINE_AA,
            )
            cv2.putText(
                image_bgr,
                f"Label: {label}",
                (50, 150),
                cv2.FONT_HERSHEY_SIMPLEX,
                1,
                (255, 255, 255),
                2,
                cv2.LINE_AA,
            )
            if collect_data:
                save_features(landmarks, exercise_type)

        elif exercise_type == "bench_press" and draw_pose:
            elbow_angle, label, start_detected, end_detected = analyze_bench_press(
                landmarks, start_detected, end_detected
            )
            cv2.putText(
                image_bgr,
                f"Elbow Angle: {elbow_angle:.2f}",
                (50, 50),
                cv2.FONT_HERSHEY_SIMPLEX,
                1,
                (255, 255, 255),
                2,
                cv2.LINE_AA,
            )
            cv2.putText(
                image_bgr,
                f"Label: {label}",
                (50, 100),
                cv2.FONT_HERSHEY_SIMPLEX,
                1,
                (255, 255, 255),
                2,
                cv2.LINE_AA,
            )
            if collect_data:
                save_features(landmarks, exercise_type)

        if draw_pose:
            mp_drawing.draw_landmarks(
                image_bgr, results.pose_landmarks, mp_pose.POSE_CONNECTIONS
            )

    return image_bgr


def save_features(landmarks, exercise):
    if exercise == "deadlift":
        back_angle, knee_angle, label, _, _ = analyze_deadlift(
            landmarks, start_detected, end_detected
        )
        data.append([back_angle, knee_angle, label])

    elif exercise == "squat":
        knee_angle, hip_angle, label, _, _ = analyze_squat(
            landmarks, start_detected, end_detected
        )
        data.append([knee_angle, hip_angle, label])

    elif exercise == "bench_press":
        elbow_angle, label, _, _ = analyze_bench_press(
            landmarks, start_detected, end_detected
        )
        data.append([elbow_angle, label])


# Function to set up the webcam video capture
def video_setup(capture_device_index=0, frame_width=1280, frame_height=720):
    capture_device = cv2.VideoCapture(capture_device_index)
    capture_device.set(3, frame_width)
    capture_device.set(4, frame_height)
    return capture_device


# Function to generate a unique filename for the recording or frame
def generate_filename(exercise, is_frame=False):
    global counter, photos_dir
    if exercise not in counter:
        counter[exercise] = 0
    counter[exercise] += 1
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    base_directory = f"{exercise}"
    video_directory = f"{base_directory}/videos"
    photos_directory = f"{base_directory}/photos"

    if not os.path.exists(video_directory):
        os.makedirs(video_directory)
    if not os.path.exists(photos_directory):
        os.makedirs(photos_directory)

    photos_dir = photos_directory

    if is_frame:
        return f"{photos_directory}/frame_{timestamp}_{counter[exercise]}.jpg"
    else:
        return f"{video_directory}/{exercise}_{timestamp}_{counter[exercise]}.avi"


# Function to start the live feed in the Tkinter window
def start_live_feed(capture, window, canvas, pose):
    global recording, video_writer
    ret, frame = capture.read()
    if ret:
        if current_exercise:
            frame = process_frame(frame, pose, current_exercise)
        else:
            frame = process_frame(frame, pose)

        if recording:
            # Save frame as an image with a unique name
            frame_filename = generate_filename(current_exercise, is_frame=True)
            cv2.imwrite(frame_filename, frame)

            if video_writer:
                video_writer.write(frame)

        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGBA)
        captured_image = Image.fromarray(image)
        photo_image = ImageTk.PhotoImage(image=captured_image)

        canvas.create_image(0, 0, anchor=tk.NW, image=photo_image)
        canvas.photo_image = photo_image  # Keep a reference to avoid garbage collection

        window.after(10, start_live_feed, capture, window, canvas, pose)


# Function to capture the initial click position and calculate offset
def start_move_window(event, window):
    window._offset_x = event.x
    window._offset_y = event.y


# Function to move the window
def move_window(event, window):
    x = event.x_root - window._offset_x
    y = event.y_root - window._offset_y
    window.geometry(f"+{x}+{y}")


# Function to close the window
def close_window(window):
    window.destroy()


# Function to setup Tkinter window and buttons
def tk_setup(
    capture,
    pose,
    dark_mode_bg_color="#333333",  # Slightly lighter dark gray for the background
    dark_mode_title_color="#181818",  # Darker gray for the title bar
    text_color="#b0b0b0",  # Light Gray
    button_bg_color="#4682b4",  # Steel Blue
    button_hover_color="#315f72",  # Darker Steel Blue
    draw_button_color="#40e0d0",  # Turquoise
    draw_button_hover_color="#2e8b87",  # Darker Turquoise
    button_start_color="#32cd32",  # Lime Green
    button_start_hover_color="#228b22",  # Darker Lime Green
    button_stop_color="#ff6347",  # Tomato
    button_stop_hover_color="#cd4f39",  # Darker Tomato
):
    window = tk.Tk()
    window.geometry("1280x800")
    window.title("Exercise Analyzer")
    window.configure(bg=dark_mode_bg_color)
    ck.set_appearance_mode("dark")
    window.overrideredirect(True)

    # Create a frame for the custom title bar
    title_bar = tk.Frame(window, bg=dark_mode_title_color, relief="raised", bd=1)
    title_bar.pack(side="top", fill="x")

    # Create a title label
    title_label = tk.Label(
        title_bar, text="Custom Title Bar", bg=dark_mode_title_color, fg="white"
    )
    title_label.pack(side="left", padx=10)

    # Create a close button
    close_button = tk.Button(
        title_bar,
        text="x",
        command=lambda: close_window(window),
        bg=dark_mode_title_color,
        fg="white",
        bd=0,
    )
    close_button.pack(side="right")

    # Bind the title bar to the move window function
    title_bar.bind("<Button-1>", lambda event: start_move_window(event, window))
    title_bar.bind("<B1-Motion>", lambda event: move_window(event, window))

    canvas = tk.Canvas(window, width=1280, height=600, highlightthickness=0, bd=0)
    canvas.pack()

    start_live_feed(capture, window, canvas, pose)

    # Frame to hold the buttons
    button_frame = tk.Frame(window)
    button_frame.configure(bg=dark_mode_bg_color)
    button_frame.pack(pady=20)  # Adjust the vertical space between canvas and buttons

    # First row of buttons
    deadlift_button = ck.CTkButton(
        button_frame,
        text="Deadlift",
        command=lambda: setup_exercise("deadlift"),
        height=40,
        width=120,
        font=("Arial", 20),
        text_color=text_color,
        fg_color=button_bg_color,
        hover_color=button_hover_color,
    )
    deadlift_button.grid(row=0, column=0, padx=10, pady=10)

    squat_button = ck.CTkButton(
        button_frame,
        text="Squat",
        command=lambda: setup_exercise("squat"),
        height=40,
        width=120,
        font=("Arial", 20),
        text_color=text_color,
        fg_color=button_bg_color,
        hover_color=button_hover_color,
    )
    squat_button.grid(row=0, column=1, padx=10, pady=10)

    bench_button = ck.CTkButton(
        button_frame,
        text="Bench Press",
        command=lambda: setup_exercise("bench_press"),
        height=40,
        width=120,
        font=("Arial", 20),
        text_color=text_color,
        fg_color=button_bg_color,
        hover_color=button_hover_color,
    )
    bench_button.grid(row=0, column=2, padx=10, pady=10)

    # Second row of buttons
    draw_button = ck.CTkButton(
        button_frame,
        text="Draw",
        command=toggle_draw_pose,
        height=40,
        width=120,
        font=("Arial", 20),
        text_color=text_color,
        fg_color=draw_button_color,
        hover_color=draw_button_hover_color,
    )
    draw_button.grid(
        row=1, column=0, padx=10, pady=20
    )  # Increased pady for more vertical space

    record_button = ck.CTkButton(
        button_frame,
        text="Start Recording",
        command=start_recording,
        height=40,
        width=150,
        font=("Arial", 20),
        text_color=text_color,
        fg_color=button_start_color,
        hover_color=button_start_hover_color,
    )
    record_button.grid(
        row=1, column=1, padx=10, pady=20
    )  # Increased pady for more vertical space

    stop_button = ck.CTkButton(
        button_frame,
        text="Stop Recording",
        command=stop_recording,
        height=40,
        width=120,
        font=("Arial", 20),
        text_color=text_color,
        fg_color=button_stop_color,
        hover_color=button_stop_hover_color,
    )
    stop_button.grid(
        row=1, column=2, padx=10, pady=20
    )  # Increased pady for more vertical space

    return window


# Function to set the current exercise type
def setup_exercise(exercise):
    global current_exercise
    current_exercise = exercise


# Function to toggle drawing of pose connections
def toggle_draw_pose():
    global draw_pose
    draw_pose = not draw_pose


# Function to start recording
def start_recording():
    global recording, video_writer
    if current_exercise:
        filename = generate_filename(current_exercise)
        recording = True
        video_writer = cv2.VideoWriter(
            filename, cv2.VideoWriter_fourcc(*"MJPG"), 10, (1280, 720)
        )
        print(f"Recording started: {filename}")


# Function to stop recording
def stop_recording():
    global recording, video_writer
    recording = False
    if video_writer:
        video_writer.release()
        video_writer = None
        print("Recording stopped")


# Main function to initialize everything
def main():
    dark_mode_title_color = "#212121"
    dark_mode_bg_color = "#2f2f2f"
    button_text_color = "#282828"  # dim grey
    button_bg_color = "#1e90ff"  # electric blue
    button_hover_color = "#104e8b"  # darker
    draw_button_color = "#00ffff"  # cyan
    draw_button_hover_color = "#008b8b"  # darker
    button_start_color = "#39ff14"  # neon green
    button_start_hover_color = "#228b22"  # darker
    button_stop_color = "#ff0000"  # red
    button_stop_hover_color = "#8b0000"  # darker

    capture = video_setup()
    with mp_pose.Pose(
        min_detection_confidence=0.5, min_tracking_confidence=0.5
    ) as pose:
        tk_window = tk_setup(
            capture,
            pose,
            dark_mode_bg_color,
            dark_mode_title_color,
            button_text_color,
            button_bg_color,
            button_hover_color,
            draw_button_color,
            draw_button_hover_color,
            button_start_color,
            button_start_hover_color,
            button_stop_color,
            button_stop_hover_color,
        )
        tk_window.mainloop()

    capture.release()
    cv2.destroyAllWindows()


if __name__ == "__main__":
    main()

Recording started: bench_press/videos/bench_press_20240727_142640_1.avi
Recording stopped


# Updating the buttons to checkboxes an' shit

In [16]:
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
import tkinter as tk
from PIL import Image, ImageTk
import customtkinter as ck
import datetime
import os

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

data = []
current_exercise = None
recording = False
video_writer = None
counter = {}
draw_pose = False
start_detected = False
end_detected = False
previous_knee_angle = None
previous_back_angle = None
failed_lift = False
photos_dir = None


# Function to calculate angles between joints
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)

    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(
        a[1] - b[1], a[0] - b[0]
    )
    angle = np.abs(radians * 180.0 / np.pi)

    if angle > 180.0:
        angle = 360 - angle

    return angle


# Function to determine if the lift is successful
def determine_label(angle, exercise):
    if exercise == "deadlift":
        return 1 if 160 <= angle <= 180 else 0
    elif exercise == "squat":
        return 1 if angle <= 90 else 0
    elif exercise == "bench_press":
        return 1 if 70 <= angle <= 110 else 0
    return 0


# Functions to analyze angles for each exercise
def analyze_deadlift(landmarks, start_detected, end_detected):
    global previous_knee_angle, previous_back_angle, failed_lift
    shoulder = [
        landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
        landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y,
    ]
    hip = [
        landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
        landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y,
    ]
    knee = [
        landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
        landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y,
    ]
    ankle = [
        landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
        landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y,
    ]

    back_angle = calculate_angle(shoulder, hip, knee)
    knee_angle = calculate_angle(hip, knee, ankle)

    if knee_angle < 45 and back_angle < 45 and not start_detected:
        start_detected = True
        end_detected = False
        failed_lift = False

    if knee_angle > 160 and back_angle > 160 and start_detected:
        end_detected = True

    if start_detected and not end_detected:
        if previous_knee_angle is not None and previous_back_angle is not None:
            if (
                knee_angle < previous_knee_angle - 20
                or back_angle < previous_back_angle - 20
            ):
                failed_lift = True

    previous_knee_angle = knee_angle
    previous_back_angle = back_angle

    label = 1 if start_detected and end_detected and not failed_lift else 0

    if end_detected:
        start_detected = False
        end_detected = False

    return back_angle, knee_angle, label, start_detected, end_detected


def analyze_squat(landmarks, start_detected, end_detected):
    hip = [
        landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
        landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y,
    ]
    knee = [
        landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
        landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y,
    ]
    ankle = [
        landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
        landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y,
    ]
    shoulder = [
        landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
        landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y,
    ]
    knee_angle = calculate_angle(hip, knee, ankle)
    hip_angle = calculate_angle(shoulder, hip, knee)

    if knee_angle > 160 and hip_angle > 160 and not start_detected:
        start_detected = True
        end_detected = False

    if knee_angle < 90 and hip_angle < 90 and start_detected:
        end_detected = True

    label = 1 if start_detected and end_detected else 0

    if end_detected:
        start_detected = False
        end_detected = False

    return knee_angle, hip_angle, label, start_detected, end_detected


def analyze_bench_press(landmarks, start_detected, end_detected):
    shoulder = [
        landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
        landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y,
    ]
    elbow = [
        landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
        landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y,
    ]
    wrist = [
        landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
        landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y,
    ]

    elbow_angle = calculate_angle(shoulder, elbow, wrist)

    if elbow_angle > 160 and not start_detected:
        start_detected = True
        end_detected = False

    if elbow_angle < 70 and start_detected:
        end_detected = True

    label = 1 if start_detected and end_detected else 0

    if end_detected:
        start_detected = False
        end_detected = False

    return elbow_angle, label, start_detected, end_detected


# Function to process the frame with MediaPipe and overlay results
def process_frame(image, pose, exercise_type=None, collect_data=False):
    global start_detected, end_detected, data
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image_rgb.flags.writeable = False
    results = pose.process(image_rgb)
    image_rgb.flags.writeable = True
    image_bgr = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR)

    if results.pose_landmarks:
        landmarks = results.pose_landmarks.landmark

        if exercise_type == "deadlift" and draw_pose:
            back_angle, knee_angle, label, start_detected, end_detected = (
                analyze_deadlift(landmarks, start_detected, end_detected)
            )
            cv2.putText(
                image_bgr,
                f"Back Angle: {back_angle:.2f}",
                (50, 50),
                cv2.FONT_HERSHEY_SIMPLEX,
                1,
                (255, 255, 255),
                2,
                cv2.LINE_AA,
            )
            cv2.putText(
                image_bgr,
                f"Knee Angle: {knee_angle:.2f}",
                (50, 100),
                cv2.FONT_HERSHEY_SIMPLEX,
                1,
                (255, 255, 255),
                2,
                cv2.LINE_AA,
            )
            cv2.putText(
                image_bgr,
                f"Label: {label}",
                (50, 150),
                cv2.FONT_HERSHEY_SIMPLEX,
                1,
                (255, 255, 255),
                2,
                cv2.LINE_AA,
            )
            if collect_data:
                save_features(landmarks, exercise_type)

        elif exercise_type == "squat" and draw_pose:
            knee_angle, hip_angle, label, start_detected, end_detected = analyze_squat(
                landmarks, start_detected, end_detected
            )
            cv2.putText(
                image_bgr,
                f"Knee Angle: {knee_angle:.2f}",
                (50, 50),
                cv2.FONT_HERSHEY_SIMPLEX,
                1,
                (255, 255, 255),
                2,
                cv2.LINE_AA,
            )
            cv2.putText(
                image_bgr,
                f"Hip Angle: {hip_angle:.2f}",
                (50, 100),
                cv2.FONT_HERSHEY_SIMPLEX,
                1,
                (255, 255, 255),
                2,
                cv2.LINE_AA,
            )
            cv2.putText(
                image_bgr,
                f"Label: {label}",
                (50, 150),
                cv2.FONT_HERSHEY_SIMPLEX,
                1,
                (255, 255, 255),
                2,
                cv2.LINE_AA,
            )
            if collect_data:
                save_features(landmarks, exercise_type)

        elif exercise_type == "bench_press" and draw_pose:
            elbow_angle, label, start_detected, end_detected = analyze_bench_press(
                landmarks, start_detected, end_detected
            )
            cv2.putText(
                image_bgr,
                f"Elbow Angle: {elbow_angle:.2f}",
                (50, 50),
                cv2.FONT_HERSHEY_SIMPLEX,
                1,
                (255, 255, 255),
                2,
                cv2.LINE_AA,
            )
            cv2.putText(
                image_bgr,
                f"Label: {label}",
                (50, 100),
                cv2.FONT_HERSHEY_SIMPLEX,
                1,
                (255, 255, 255),
                2,
                cv2.LINE_AA,
            )
            if collect_data:
                save_features(landmarks, exercise_type)

        if draw_pose:
            mp_drawing.draw_landmarks(
                image_bgr, results.pose_landmarks, mp_pose.POSE_CONNECTIONS
            )

    return image_bgr


def save_features(landmarks, exercise):
    if exercise == "deadlift":
        back_angle, knee_angle, label, _, _ = analyze_deadlift(
            landmarks, start_detected, end_detected
        )
        data.append([back_angle, knee_angle, label])

    elif exercise == "squat":
        knee_angle, hip_angle, label, _, _ = analyze_squat(
            landmarks, start_detected, end_detected
        )
        data.append([knee_angle, hip_angle, label])

    elif exercise == "bench_press":
        elbow_angle, label, _, _ = analyze_bench_press(
            landmarks, start_detected, end_detected
        )
        data.append([elbow_angle, label])


# Function to set up the webcam video capture
def video_setup(capture_device_index=0, frame_width=1280, frame_height=720):
    capture_device = cv2.VideoCapture(capture_device_index)
    capture_device.set(3, frame_width)
    capture_device.set(4, frame_height)
    return capture_device


# Function to generate a unique filename for the recording or frame
def generate_filename(exercise, is_frame=False):
    global counter, photos_dir
    if exercise not in counter:
        counter[exercise] = 0
    counter[exercise] += 1
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    base_directory = f"{exercise}"
    video_directory = f"{base_directory}/videos"
    photos_directory = f"{base_directory}/photos"

    if not os.path.exists(video_directory):
        os.makedirs(video_directory)
    if not os.path.exists(photos_directory):
        os.makedirs(photos_directory)

    photos_dir = photos_directory

    if is_frame:
        return f"{photos_directory}/frame_{timestamp}_{counter[exercise]}.jpg"
    else:
        return f"{video_directory}/{exercise}_{timestamp}_{counter[exercise]}.avi"


# Function to start the live feed in the Tkinter window
def start_live_feed(capture, window, canvas, pose):
    global recording, video_writer
    ret, frame = capture.read()
    if ret:
        if current_exercise:
            frame = process_frame(frame, pose, current_exercise)
        else:
            frame = process_frame(frame, pose)

        if recording:
            frame_filename = generate_filename(current_exercise, is_frame=True)
            cv2.imwrite(frame_filename, frame)

            if video_writer:
                video_writer.write(frame)

        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGBA)
        captured_image = Image.fromarray(image)
        photo_image = ImageTk.PhotoImage(image=captured_image)

        canvas.create_image(0, 0, anchor=tk.NW, image=photo_image)
        canvas.photo_image = photo_image

        window.after(10, start_live_feed, capture, window, canvas, pose)


# Function to capture the initial click position and calculate offset
def start_move_window(event, window):
    window._offset_x = event.x
    window._offset_y = event.y


# Function to move the window
def move_window(event, window):
    x = event.x_root - window._offset_x
    y = event.y_root - window._offset_y
    window.geometry(f"+{x}+{y}")


# Function to close the window
def close_window(window):
    window.destroy()


# Function to set the current exercise type
def setup_exercise(exercise):
    global current_exercise
    if exercise == "deadlift":
        squat_var.set(False)
        bench_var.set(False)
    elif exercise == "squat":
        deadlift_var.set(False)
        bench_var.set(False)
    elif exercise == "bench_press":
        deadlift_var.set(False)
        squat_var.set(False)

    current_exercise = exercise


# Function to toggle drawing of pose connections
def toggle_draw_pose():
    global draw_pose
    draw_pose = not draw_pose


# Variable to track recording state
recording = False


def toggle_recording():
    global recording, video_writer
    if recording:
        recording = False
        if video_writer:
            video_writer.release()
            video_writer = None
        record_button.configure(text="Start Recording", fg_color="green")
        print("Recording stopped")
    else:
        if current_exercise:
            filename = generate_filename(current_exercise)
            recording = True
            video_writer = cv2.VideoWriter(
                filename, cv2.VideoWriter_fourcc(*"MJPG"), 10, (1280, 720)
            )
            record_button.configure(text="Stop Recording", fg_color="red")
            print(f"Recording started: {filename}")


# Function to setup Tkinter window and buttons
def tk_setup(
    capture,
    pose,
    dark_mode_bg_color="#333333",
    dark_mode_title_color="#181818",
    text_color="#b0b0b0",
    button_bg_color="#4682b4",
    button_hover_color="#315f72",
    draw_button_color="#40e0d0",
    draw_button_hover_color="#2e8b87",
    button_start_color="#32cd32",
    button_start_hover_color="#228b22",
    button_stop_color="#ff6347",
    button_stop_hover_color="#cd4f39",
):
    window = tk.Tk()
    window.geometry("1280x800")
    window.title("Exercise Analyzer")
    window.configure(bg=dark_mode_bg_color)
    ck.set_appearance_mode("dark")
    window.overrideredirect(True)

    # Create a frame for the custom title bar
    title_bar = tk.Frame(window, bg=dark_mode_title_color, relief="raised", bd=1)
    title_bar.pack(side="top", fill="x")

    # Create a title label
    title_label = tk.Label(
        title_bar, text="Custom Title Bar", bg=dark_mode_title_color, fg="white"
    )
    title_label.pack(side="left", padx=10)

    # Create a close button
    close_button = tk.Button(
        title_bar,
        text="x",
        command=lambda: close_window(window),
        bg=dark_mode_title_color,
        fg="white",
        bd=0,
    )
    close_button.pack(side="right")

    # Bind the title bar to the move window function
    title_bar.bind("<Button-1>", lambda event: start_move_window(event, window))
    title_bar.bind("<B1-Motion>", lambda event: move_window(event, window))

    canvas = tk.Canvas(window, width=1280, height=600, highlightthickness=0, bd=0)
    canvas.pack()

    start_live_feed(capture, window, canvas, pose)

    # Frame to hold the buttons
    button_frame = tk.Frame(window)
    button_frame.configure(bg=dark_mode_bg_color)
    button_frame.pack(pady=20)

    # Boolean variables to track the state of checkboxes
    global deadlift_var, squat_var, bench_var, draw_var
    deadlift_var = tk.BooleanVar()
    squat_var = tk.BooleanVar()
    bench_var = tk.BooleanVar()
    draw_var = tk.BooleanVar()

    # Define custom styles
    checkbox_font = ("Arial", 20)  # Font size
    checkbox_bg = "#2f2f2f"  # Background color
    checkbox_fg = "#b2beb5"  # Text color
    select_color = "#2f2f2f" # checkbox color
    checkbox_width = 15      # Uniform width for checkboxes
    # Create checkboxes with custom styles and uniform width
    deadlift_checkbox = tk.Checkbutton(
        button_frame,
        text="Deadlift",
        variable=deadlift_var,
        command=lambda: setup_exercise("deadlift"),
        font=checkbox_font,
        bg=checkbox_bg,
        fg=checkbox_fg,
        selectcolor=select_color,
        width=checkbox_width
    )
    squat_checkbox = tk.Checkbutton(
        button_frame,
        text="Squat",
        variable=squat_var,
        command=lambda: setup_exercise("squat"),
        font=checkbox_font,
        bg=checkbox_bg,
        fg=checkbox_fg,
        selectcolor=select_color,
        width=checkbox_width
    )
    bench_checkbox = tk.Checkbutton(
        button_frame,
        text="Bench Press",
        variable=bench_var,
        command=lambda: setup_exercise("bench_press"),
        font=checkbox_font,
        bg=checkbox_bg,
        fg=checkbox_fg,
        selectcolor=select_color,
        width=checkbox_width
    )
    draw_checkbox = tk.Checkbutton(
        button_frame,
        text="Draw",
        variable=draw_var,
        command=toggle_draw_pose,
        font=checkbox_font,
        bg=checkbox_bg,
        fg=checkbox_fg,
        selectcolor=select_color,
        width=checkbox_width
    )

    # Arrange checkboxes in a single line using grid layout
    deadlift_checkbox.grid(row=0, column=0, padx=10, pady=10, sticky="ew")
    squat_checkbox.grid(row=0, column=1, padx=10, pady=10, sticky="ew")
    bench_checkbox.grid(row=0, column=2, padx=10, pady=10, sticky="ew")
    draw_checkbox.grid(row=0, column=3, padx=10, pady=10, sticky="ew")

    # Configure grid to distribute space evenly
    button_frame.grid_columnconfigure(0, weight=1)
    button_frame.grid_columnconfigure(1, weight=1)
    button_frame.grid_columnconfigure(2, weight=1)
    button_frame.grid_columnconfigure(3, weight=1)

    # Record toggle button
    global record_button
    record_button = ck.CTkButton(
        button_frame,
        text="Start Recording",
        command=toggle_recording,
        height=40,
        width=150,
        font=("Arial", 32),
        text_color=text_color,
        fg_color=button_start_color
    )
    record_button.grid(row=1, column=1, columnspan=2, padx=10, pady=20, sticky="ew")

    return window


# Main function to initialize everything
def main():
    dark_mode_title_color = "#212121"
    dark_mode_bg_color = "#2f2f2f"
    button_text_color = "#282828"


    capture = video_setup()
    with mp_pose.Pose(
        min_detection_confidence=0.5, min_tracking_confidence=0.5
    ) as pose:
        tk_window = tk_setup(
            capture,
            pose,
            dark_mode_bg_color,
            dark_mode_title_color,
            button_text_color,

        )
        tk_window.mainloop()

    capture.release()
    cv2.destroyAllWindows()


if __name__ == "__main__":
    main()

Recording started: deadlift/videos/deadlift_20240727_145743_1.avi
Recording stopped
Recording started: bench_press/videos/bench_press_20240727_145755_1.avi
Recording stopped
Recording started: squat/videos/squat_20240727_145810_1.avi
Recording stopped
Recording started: squat/videos/squat_20240727_145827_38.avi
Recording stopped
Recording started: bench_press/videos/bench_press_20240727_145902_38.avi
Recording stopped
Recording started: bench_press/videos/bench_press_20240727_145951_61.avi
Recording stopped
Recording started: deadlift/videos/deadlift_20240727_150015_24.avi
Recording stopped


# First draft of reading manual photo directories
* This loops through all of the directories that we put the photos in manually
    * there directories have this structure:
        * `{exercise_name}/start` 
        * `{exercist_name}/end`
        * `{exercise_name}/in_between`
* We finally have pd being used

In [ ]:
# first pass with annotating - just ok
import cv2
import mediapipe as mp
import pandas as pd
import os

# Initialize MediaPipe Pose and Drawing utilities
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose(static_image_mode=True)

# Define directories
directories = {
    "deadlift_start": "deadlift/start",
    "deadlift_end": "deadlift/end",
    "deadlift_in_between": "deadlift/in_between",
    "squat_start": "squat/start",
    "squat_end": "squat/end",
    "squat_in_between": "squat/in_between",
    "bench_press_start": "bench_press/start",
    "bench_press_end": "bench_press/end",
    "bench_press_in_between": "bench_press/in_between",
}


# Function to extract and draw key points on an image
def process_image(image, output_path):
    # Process the image to detect key points
    results = pose.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

    # Draw pose landmarks if detected
    if results.pose_landmarks:
        mp_drawing.draw_landmarks(
            image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS
        )

    # Save the annotated image
    cv2.imwrite(output_path, image)

    # Extract key points
    if results.pose_landmarks:
        landmarks = results.pose_landmarks.landmark
        keypoints = [(landmark.x, landmark.y) for landmark in landmarks]
        return keypoints
    return None


# List to store data
data = []

# Process each directory
for label, dir_path in directories.items():
    if not os.path.exists(dir_path):
        print(f"Directory {dir_path} does not exist.")
        continue

    for filename in os.listdir(dir_path):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            image_path = os.path.join(dir_path, filename)
            output_path = os.path.join(dir_path, "annotated_" + filename)

            image = cv2.imread(image_path)
            keypoints = process_image(image, output_path)

            if keypoints:
                flat_keypoints = [coord for point in keypoints for coord in point]
                data.append([filename] + flat_keypoints + [label])
            else:
                print(f"No keypoints detected in {filename}.")

# Create a DataFrame
columns = (
    ["filename"]
    + [f"x{i}" for i in range(33)]
    + [f"y{i}" for i in range(33)]
    + ["label"]
)
df = pd.DataFrame(data, columns=columns)

# Save DataFrame to CSV
output_csv = "keypoints_labels.csv"
df.to_csv(output_csv, index=False)

print(f"Keypoints extracted and saved to {output_csv}.")

# Release resources
pose.close()

# This is a check for annotated folders to see if they exist
* I think this is because I kept changing the directories and structure that I wanted to use
* We will figure out when I start deleting shit and something breaks

In [ ]:
# quick check for cleanup
import os

directories = {
    "deadlift_start": "deadlift/start",
    "deadlift_end": "deadlift/end",
    "deadlift_in_between": "deadlift/in_between",
    "squat_start": "squat/start",
    "squat_end": "squat/end",
    "squat_in_between": "squat/in_between",
    "bench_press_start": "bench_press/start",
    "bench_press_end": "bench_press/end",
    "bench_press_in_between": "bench_press/in_between",
}

annotated_dirs = []

for key, dir_path in directories.items():
    contains_annotated = False
    if os.path.exists(dir_path):
        for root, dirs, files in os.walk(dir_path):
            for name in files + dirs:
                if "annotated" in name:
                    contains_annotated = True
                    annotated_dirs.append(dir_path)
                    break
            if contains_annotated:
                break

    print(
        f"{key}: {'Contains' if contains_annotated else 'Does not contain'} 'annotated'"
    )

if annotated_dirs:
    print("\nDirectories containing 'annotated':")
    for dir_path in annotated_dirs:
        print(dir_path)
else:
    print("\nGood to go! No directories contain 'annotated'")

# This was trials of different preprocessing
* I should have made different code blocks
    * I don't know if the uncommented method is the 'best' or if I just got tired and gave up with manually changing shit all the time

In [ ]:
# second pass at annotating
import cv2
import mediapipe as mp
import pandas as pd
import os

# Initialize MediaPipe Pose and Drawing utilities
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.5, min_tracking_confidence=0.5)

# Define directories
directories = {
    'deadlift_start': 'deadlift/start',
    'deadlift_end': 'deadlift/end',
    'deadlift_in_between': 'deadlift/in_between',
    'squat_start': 'squat/start',
    'squat_end': 'squat/end',
    'squat_in_between': 'squat/in_between',
    'bench_press_start': 'bench_press/start',
    'bench_press_end': 'bench_press/end',
    'bench_press_in_between': 'bench_press/in_between'
}

# Function to preprocess the image
def preprocess_image(image):
    # Convert to grayscale and back to BGR
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR)

    # Resize the image to a standard size
    image = cv2.resize(image, (640, 480))

    # Apply Gaussian blur
    image = cv2.GaussianBlur(image, (5, 5), 0)

    # Adjust brightness and contrast
    alpha = 1.2  # Contrast control (1.0-3.0)
    beta = 20    # Brightness control (0-100)
    image = cv2.convertScaleAbs(image, alpha=alpha, beta=beta)

    return image

# Function to detect if an image already has pose annotations by checking for red dots
# def has_existing_annotations(image):
#     hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
#     # Define the red color range for MediaPipe annotations
#     lower_red = np.array([0, 50, 50])
#     upper_red = np.array([10, 255, 255])
#     mask = cv2.inRange(hsv_image, lower_red, upper_red)
#     red_pixels = cv2.countNonZero(mask)
#     return red_pixels > 500  # Adjust threshold as needed

# Function to detect if an image already has pose annotations by checking for red and white dots
# def has_existing_annotations(image):
#     hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
#     # Define the red color range for MediaPipe annotations
#     lower_red = np.array([0, 50, 50])
#     upper_red = np.array([10, 255, 255])
#     mask_red = cv2.inRange(hsv_image, lower_red, upper_red)
#     red_pixels = cv2.countNonZero(mask_red)

#     # Define the white color range for MediaPipe connections
#     lower_white = np.array([0, 0, 200])
#     upper_white = np.array([180, 20, 255])
#     mask_white = cv2.inRange(hsv_image, lower_white, upper_white)
#     white_pixels = cv2.countNonZero(mask_white)
# Check both red and white pixels
# return (red_pixels > 500) or (white_pixels > 500)  # Adjust thresholds as needed

def has_existing_annotations(image):
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    lower_red = np.array([0, 50, 50])
    upper_red = np.array([10, 255, 255])
    mask_red = cv2.inRange(hsv_image, lower_red, upper_red)
    red_pixels = cv2.countNonZero(mask_red)

    lower_white = np.array([0, 0, 200])
    upper_white = np.array([180, 20, 255])
    mask_white = cv2.inRange(hsv_image, lower_white, upper_white)
    white_pixels = cv2.countNonZero(mask_white)

    contours_red, _ = cv2.findContours(mask_red, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours_white, _ = cv2.findContours(
        mask_white, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE
    )

    small_red_regions = sum(1 for cnt in contours_red if cv2.contourArea(cnt) < 500)
    small_white_regions = sum(1 for cnt in contours_white if cv2.contourArea(cnt) < 500)

    return (red_pixels > 500 and small_red_regions > 5) or (
        white_pixels > 500 and small_white_regions > 5
    )


# Function to extract and draw key points on an image
def process_image(image, output_path):
    image = preprocess_image(image)  # Preprocess the image if no existing annotations
    if not has_existing_annotations(image):
        results = pose.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(
                image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS
            )
        cv2.imwrite(output_path, image)
        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark
            keypoints = [(landmark.x, landmark.y) for landmark in landmarks]
            return keypoints
    else:
        cv2.imwrite(output_path, image)
    return None


# List to store data
data = []

# Process each directory
for label, dir_path in directories.items():
    if not os.path.exists(dir_path):
        print(f"Directory {dir_path} does not exist.")
        continue
    
    annotated_dir = os.path.join(dir_path, 'annotated_2_')
    os.makedirs(annotated_dir, exist_ok=True)
    
    for filename in os.listdir(dir_path):
        if filename.endswith('.jpg') or filename.endswith('.png'):
            image_path = os.path.join(dir_path, filename)
            output_path = os.path.join(annotated_dir, filename)
            
            image = cv2.imread(image_path)
            keypoints = process_image(image, output_path)
            
            if keypoints:
                flat_keypoints = [coord for point in keypoints for coord in point]
                data.append([filename] + flat_keypoints + [label])
            else:
                print(f"No keypoints detected in {filename}.")

# Create a DataFrame
columns = ['filename'] + [f'x{i}' for i in range(33)] + [f'y{i}' for i in range(33)] + ['label']
df = pd.DataFrame(data, columns=columns)

# Save DataFrame to CSV
output_csv = 'keypoints_labels.csv'
df.to_csv(output_csv, index=False)

print(f"Keypoints extracted and saved to {output_csv}.")

# Release resources
pose.close()

# I think this was one of the preprocessing tests and I think I just started using the above block to do everything 
* Because I am an idiot and lazy (and I thought it would be 'easy' to go back and figure out what I have done lol)

In [ ]:
# debugging barbell issues
import cv2
import numpy as np
import os

# Define directories
directories = {
    "deadlift_start": "deadlift/start",
    "deadlift_end": "deadlift/end",
    "deadlift_in_between": "deadlift/in_between",
}


# Function to preprocess the image
def preprocess_image(image):
    # Convert to grayscale and back to BGR
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR)

    # Resize the image to a standard size
    image = cv2.resize(image, (640, 480))

    # Apply Gaussian blur
    image = cv2.GaussianBlur(image, (5, 5), 0)

    # Adjust brightness and contrast
    alpha = 1.2  # Contrast control (1.0-3.0)
    beta = 20  # Brightness control (0-100)
    image = cv2.convertScaleAbs(image, alpha=alpha, beta=beta)

    return image


# Function to detect if an image already has pose annotations by checking for red dots
def has_existing_annotations(image):
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    # Define the red color range for MediaPipe annotations
    lower_red = np.array([0, 50, 50])
    upper_red = np.array([10, 255, 255])
    mask = cv2.inRange(hsv_image, lower_red, upper_red)
    red_pixels = cv2.countNonZero(mask)
    return red_pixels > 500  # Adjust threshold as needed


# Process each directory and save images
for label, dir_path in directories.items():
    if not os.path.exists(dir_path):
        print(f"Directory {dir_path} does not exist.")
        continue

    preprocessed_dir = os.path.join(dir_path, "preprocessed")
    annotated_dir = os.path.join(dir_path, "annotated")
    os.makedirs(preprocessed_dir, exist_ok=True)
    os.makedirs(annotated_dir, exist_ok=True)

    for filename in os.listdir(dir_path):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            image_path = os.path.join(dir_path, filename)

            image = cv2.imread(image_path)

            # Save preprocessed image
            preprocessed_image = preprocess_image(image)
            preprocessed_output_path = os.path.join(preprocessed_dir, filename)
            cv2.imwrite(preprocessed_output_path, preprocessed_image)

            # Check for existing annotations
            if has_existing_annotations(image):
                annotated_output_path = os.path.join(annotated_dir, filename)
                cv2.imwrite(annotated_output_path, image)
            else:
                print(f"No existing annotations found in {filename}.")

print("Processing complete.")

# I said this next block was my best for getting poses drawn onto non-posed images
* I don't know if this is true
    * if true then I didn't check anything other than my benchpress photos b/c after coming back deadlift and squats didn't have images posed that weren't already posed from the video recording images that were in the folders

In [ ]:
#########################
# winner annotation pass #
###########################
import cv2
import mediapipe as mp
import pandas as pd
import numpy as np
import os

# Initialize MediaPipe Pose and Drawing utilities
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose(
    static_image_mode=True, min_detection_confidence=0.7, min_tracking_confidence=0.7
)

# Define directories
directories = {
    "deadlift_start": "deadlift/start",
    "deadlift_end": "deadlift/end",
    "deadlift_in_between": "deadlift/in_between",
    "squat_start": "squat/start",
    "squat_end": "squat/end",
    "squat_in_between": "squat/in_between",
    "bench_press_start": "bench_press/start",
    "bench_press_end": "bench_press/end",
    "bench_press_in_between": "bench_press/in_between",
}


# Function to preprocess the image
def preprocess_image(image):
    # Simplified preprocessing without grayscale and brightness/contrast adjustment
    image = cv2.resize(image, (640, 480))
    image = cv2.GaussianBlur(image, (5, 5), 0)
    return image


# Function to detect if an image already has pose annotations
def has_existing_annotations(image):
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Define the red color range for MediaPipe annotations
    lower_red = np.array([0, 50, 50])
    upper_red = np.array([10, 255, 255])
    mask_red = cv2.inRange(hsv_image, lower_red, upper_red)
    red_pixels = cv2.countNonZero(mask_red)

    # Define the white color range for MediaPipe connections
    lower_white = np.array([0, 0, 200])
    upper_white = np.array([180, 20, 255])
    mask_white = cv2.inRange(hsv_image, lower_white, upper_white)
    white_pixels = cv2.countNonZero(mask_white)

    # Structural check: Ensure that detected regions are small and clustered
    contours_red, _ = cv2.findContours(mask_red, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours_white, _ = cv2.findContours(
        mask_white, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE
    )

    small_red_regions = sum(1 for cnt in contours_red if cv2.contourArea(cnt) < 500)
    small_white_regions = sum(1 for cnt in contours_white if cv2.contourArea(cnt) < 500)

    # Debug information
    print(f"Red pixels: {red_pixels}, small red regions: {small_red_regions}")
    print(f"White pixels: {white_pixels}, small white regions: {small_white_regions}")

    # Adjusted threshold conditions
    red_condition = red_pixels > 2000 and 10 <= small_red_regions <= 30
    white_condition = white_pixels > 2000 and 10 <= small_white_regions <= 30

    return red_condition or white_condition


# Function to extract and draw key points on an image
def process_image(image, output_path):
    preprocessed_image = preprocess_image(image)  # Preprocess the image
    print(f"Processing image: {output_path}")  # Debug information

    preprocessed_output_path = output_path.replace("annotated_2_", "preprocessed")
    print(
        f"Saving preprocessed image to: {preprocessed_output_path}"
    )  # Debug information
    cv2.imwrite(preprocessed_output_path, preprocessed_image)

    if not has_existing_annotations(preprocessed_image):
        results = pose.process(cv2.cvtColor(preprocessed_image, cv2.COLOR_BGR2RGB))
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(
                preprocessed_image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS
            )
        cv2.imwrite(output_path, preprocessed_image)
        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark
            keypoints = [(landmark.x, landmark.y) for landmark in landmarks]
            return keypoints
    else:
        print(f"Existing annotations detected for image: {output_path}")
    return None


# List to store data
data = []

# Process each directory
for label, dir_path in directories.items():
    if not os.path.exists(dir_path):
        print(f"Directory {dir_path} does not exist.")
        continue

    preprocessed_dir = os.path.join(dir_path, "preprocessed")
    annotated_dir = os.path.join(dir_path, "annotated_2_")
    os.makedirs(preprocessed_dir, exist_ok=True)
    os.makedirs(annotated_dir, exist_ok=True)

    for filename in os.listdir(dir_path):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            image_path = os.path.join(dir_path, filename)
            output_path = os.path.join(annotated_dir, filename)

            image = cv2.imread(image_path)
            if not has_existing_annotations(image):
                keypoints = process_image(image, output_path)
                if keypoints:
                    flat_keypoints = [coord for point in keypoints for coord in point]
                    data.append([filename] + flat_keypoints + [label])
                else:
                    print(f"No keypoints detected in {filename}.")
            else:
                print(f"Skipping {filename} as it already has annotations.")

# Create a DataFrame
columns = (
    ["filename"]
    + [f"x{i}" for i in range(33)]
    + [f"y{i}" for i in range(33)]
    + ["label"]
)
df = pd.DataFrame(data, columns=columns)

# Save DataFrame to CSV
output_csv = "keypoints_labels.csv"
df.to_csv(output_csv, index=False)

print(f"Keypoints extracted and saved to {output_csv}.")

# Release resources
pose.close()

# This was a check to ensure images were getting pose drawn
* This was for an issue I was running into where pics with barbell plates not drawing the pose
* This is also why there are circular blocks and I don't remember what the end/winner annotation and preprocessing functions were

In [ ]:
# debugging
import cv2
import mediapipe as mp
import numpy as np

# Load the image
image_path_with_plates = (
    "bench_press/end/frame_20240718_120908_1515.jpg"  # Update this path
)
image_with_plates = cv2.imread(image_path_with_plates)

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose(
    static_image_mode=True, min_detection_confidence=0.7, min_tracking_confidence=0.7
)


# Preprocess the image
def preprocess_image(image):
    # Remove grayscale conversion and brightness/contrast adjustment
    image = cv2.resize(image, (640, 480))
    image = cv2.GaussianBlur(image, (5, 5), 0)
    return image


preprocessed_image_with_plates = preprocess_image(image_with_plates)

# Detect keypoints
results_with_plates = pose.process(
    cv2.cvtColor(preprocessed_image_with_plates, cv2.COLOR_BGR2RGB)
)

# Print keypoints if detected
if results_with_plates.pose_landmarks:
    print("Keypoints detected:")
    for idx, landmark in enumerate(results_with_plates.pose_landmarks.landmark):
        print(f"Landmark {idx}: ({landmark.x}, {landmark.y}, {landmark.z})")
else:
    print("No keypoints detected.")

# Draw keypoints on the image
if results_with_plates.pose_landmarks:
    mp_drawing.draw_landmarks(
        preprocessed_image_with_plates,
        results_with_plates.pose_landmarks,
        mp_pose.POSE_CONNECTIONS,
    )

# Save the processed image for visualization
processed_image_with_plates_path = (
    "bench_press/end/debug/processed_with_plates.jpg"  # Update this path
)
cv2.imwrite(processed_image_with_plates_path, preprocessed_image_with_plates)

print("Processed image saved at:", processed_image_with_plates_path)

# Install Dependencies
* had some issues so I made two pip installs

In [ ]:
pip install tensorflow

# Install Dependencies
* had some issues so I made two pip installs

In [ ]:
pip install scikit-learn

# Some type of shit for training the model
* Probably not the first, but definitely not the last (I think)

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Dense,
    Conv2D,
    Flatten,
    MaxPooling2D,
    Dropout,
    BatchNormalization,
)
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Load the data from CSV
df = pd.read_csv("keypoints_labels.csv")

# Separate features (X) and labels (y)
X = df.drop(["filename", "label"], axis=1).values
y = df["label"].values

# Normalize the keypoints
X = X / 640.0  # Assuming the image dimensions are 640x480, adjust if different

# Encode labels into one-hot format
unique_labels = np.unique(y)
label_map = {label: index for index, label in enumerate(unique_labels)}
y = np.array([label_map[label] for label in y])
y = to_categorical(y)

# Verify the shape of y
print(f"Shape of y after encoding: {y.shape}")

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Reshape the data for the CNN input
X_train = X_train.reshape(-1, 33, 2, 1)  # 33 keypoints, 2 coordinates (x, y), 1 channel
X_test = X_test.reshape(-1, 33, 2, 1)

# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=10, width_shift_range=0.1, height_shift_range=0.1, zoom_range=0.1
)
datagen.fit(X_train)

# Build the custom CNN model
model = Sequential(
    [
        Conv2D(
            32,
            (3, 2),
            activation="relu",
            padding="same",
            input_shape=(33, 2, 1),
            kernel_regularizer=l2(0.01),
        ),
        BatchNormalization(),
        MaxPooling2D((2, 1)),
        Dropout(0.3),
        Conv2D(
            64, (3, 2), activation="relu", padding="same", kernel_regularizer=l2(0.01)
        ),
        BatchNormalization(),
        MaxPooling2D((2, 1)),
        Dropout(0.3),
        Flatten(),
        Dense(128, activation="relu", kernel_regularizer=l2(0.01)),
        BatchNormalization(),
        Dropout(0.5),
        Dense(y.shape[1], activation="softmax"),
    ]
)

# Compile the model
optimizer = Adam(learning_rate=0.001)
model.compile(
    optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"]
)

# Callbacks
early_stopping = EarlyStopping(
    monitor="val_loss", patience=10, restore_best_weights=True
)

reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.2, patience=5, min_lr=0.0001)

# Train the model with data augmentation
history = model.fit(
    datagen.flow(X_train, y_train, batch_size=32),
    epochs=50,
    validation_data=(X_test, y_test),
    callbacks=[early_stopping, reduce_lr],
)

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_acc:.4f}")

# Save the trained model to a file
model.save("exercise_pose_cnn_model_custom_simple_v4.h5")

# Small check to see if gpu was being used
* it wasn't
    * still need to fix this but it is boring

In [ ]:
import tensorflow as tf


def check_gpu():
    gpus = tf.config.list_physical_devices("GPU")
    if gpus:
        print("GPUs found:")
        for gpu in gpus:
            print(f"- {gpu}")
    else:
        print("No GPU found.")


if __name__ == "__main__":
    print(tf.__version__)
    check_gpu()

# An enhanced training model
* It fails to do high epochs > 50 most of the time
* I think this is the last one I was working on and deemed to be the best, but was running into the cpu bottleneck which led to the gpu shit

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Dense,
    Conv2D,
    Flatten,
    MaxPooling2D,
    Dropout,
    BatchNormalization,
)
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Check if GPU is available and set memory growth
gpus = tf.config.list_physical_devices("GPU")

print(f"GPUS: {gpus}")
if gpus:
    try:
        for gpu in gpus:
            tf.config.set_memory_growth(gpu, True)
        tf.config.set_visible_devices(gpus[0], "GPU")
        print(f"Using GPU: {tf.config.get_visible_devices('GPU')}")
    except RuntimeError as e:
        print(e)
else:
    print("No GPU found. Using CPU.")
# Load the data from CSV
df = pd.read_csv("keypoints_labels.csv")

# Separate features (X) and labels (y)
X = df.drop(["filename", "label"], axis=1).values
y = df["label"].values

# Normalize the keypoints
X = X / 640.0  # Assuming the image dimensions are 640x480, adjust if different

# Encode labels into one-hot format
unique_labels = np.unique(y)
label_map = {label: index for index, label in enumerate(unique_labels)}
y = np.array([label_map[label] for label in y])
y = to_categorical(y)

# Verify the shape of y
print(f"Shape of y after encoding: {y.shape}")

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Reshape the data for the CNN input
X_train = X_train.reshape(-1, 33, 2, 1)  # 33 keypoints, 2 coordinates (x, y), 1 channel
X_test = X_test.reshape(-1, 33, 2, 1)

# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
)
datagen.fit(X_train)

# Build the custom CNN model
model = Sequential(
    [
        Conv2D(
            32,
            (3, 2),
            activation="relu",
            padding="same",
            input_shape=(33, 2, 1),
            kernel_regularizer=l2(0.01),
        ),
        BatchNormalization(),
        MaxPooling2D((2, 1)),
        Dropout(0.3),
        Conv2D(
            64, (3, 2), activation="relu", padding="same", kernel_regularizer=l2(0.01)
        ),
        BatchNormalization(),
        MaxPooling2D((2, 1)),
        Dropout(0.3),
        Conv2D(
            128, (3, 2), activation="relu", padding="same", kernel_regularizer=l2(0.01)
        ),
        BatchNormalization(),
        MaxPooling2D((2, 1)),
        Dropout(0.3),
        Flatten(),
        Dense(256, activation="relu", kernel_regularizer=l2(0.01)),
        BatchNormalization(),
        Dropout(0.5),
        Dense(y.shape[1], activation="softmax"),
    ]
)

# Compile the model
optimizer = Adam(learning_rate=0.001)
model.compile(
    optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"]
)

# Callbacks
early_stopping = EarlyStopping(
    monitor="val_loss", patience=10, restore_best_weights=True
)

reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.2, patience=5, min_lr=0.0001)

# Train the model with data augmentation
history = model.fit(
    datagen.flow(X_train, y_train, batch_size=32),
    epochs=100,  # Reduced epochs for smaller dataset
    validation_data=(X_test, y_test),
    callbacks=[early_stopping, reduce_lr],
)

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_acc:.4f}")

# Save the trained model to a file
model.save("exercise_pose_cnn_model_custom_enhanced_v2.h5")

# This is the checking of the model
* it is currently shit
    * everything is a deadlift right now

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
from tensorflow.keras.models import load_model
import os
import pandas as pd

# Load the trained model
model = load_model("exercise_pose_cnn_model_custom_enhanced.h5")

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.7)


# Function to preprocess the image
def preprocess_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.resize(image, (640, 480))
    image = cv2.GaussianBlur(image, (5, 5), 0)

    results = pose.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    if results.pose_landmarks:
        landmarks = results.pose_landmarks.landmark
        keypoints = [(landmark.x, landmark.y) for landmark in landmarks]
        flat_keypoints = [coord for point in keypoints for coord in point]
        return np.array(flat_keypoints).reshape(1, 33, 2, 1)
    else:
        print("No keypoints detected in image:", image_path)
        return None


# Function to predict the class of a given image
def predict_class(image_path, model, label_map):
    X_test_image = preprocess_image(image_path)
    if X_test_image is not None:
        predictions = model.predict(X_test_image)
        predicted_class_index = np.argmax(predictions)
        if predicted_class_index in label_map:
            predicted_class_label = label_map[predicted_class_index]
            return predicted_class_label
        else:
            return f"Predicted class index {predicted_class_index} is not in label map."
    else:
        return "Preprocessing failed, cannot make prediction."


# Function to get predictions for all images in a folder and save to CSV using pandas
def predict_folder_to_csv_pandas(folder_path, model, label_map, csv_path):
    data = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".jpg") or filename.endswith(
            ".png"
        ):  # Add more extensions if needed
            image_path = os.path.join(folder_path, filename)
            predicted_label = predict_class(image_path, model, label_map)
            data.append([filename, predicted_label])
            print(f"Image: {filename}, Predicted class: {predicted_label}")

    df = pd.DataFrame(data, columns=["Image Name", "Prediction"])
    df.to_csv(csv_path, index=False)


# Example usage
folder_path = "F:/deadlift/photos/"
csv_path = "deadlift_exercise_pose_cnn_model_custom_enhanced.csv"
label_map = {
    0: "deadlift_start",
    1: "deadlift_end",
    2: "deadlift_in_between",
    3: "squat_start",
    4: "squat_end",
    5: "squat_in_between",
    6: "bench_press_start",
    7: "bench_press_end",
    8: "bench_press_in_between",
}

predict_folder_to_csv_pandas(folder_path, model, label_map, csv_path)

# Nothing to be seen here

In [ ]:
# For future me to look at when I decide I want to jump into refactoring
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
import tkinter as tk
from PIL import Image, ImageTk
import customtkinter as ck
import datetime
import os
from typing import Tuple, List, Optional

# Configuration
CONFIG = {
    "CAPTURE_DEVICE_INDEX": 0,
    "FRAME_WIDTH": 1280,
    "FRAME_HEIGHT": 720,
    "DARK_MODE_BG_COLOR": "#2f2f2f",
    "DARK_MODE_TITLE_COLOR": "#212121",
    "BUTTON_TEXT_COLOR": "#282828",
    "BUTTON_BG_COLOR": "#1e90ff",
    "BUTTON_HOVER_COLOR": "#104e8b",
    "DRAW_BUTTON_COLOR": "#00ffff",
    "DRAW_BUTTON_HOVER_COLOR": "#008b8b",
    "BUTTON_START_COLOR": "#39ff14",
    "BUTTON_START_HOVER_COLOR": "#228b22",
    "BUTTON_STOP_COLOR": "#ff0000",
    "BUTTON_STOP_HOVER_COLOR": "#8b0000",
}

class ExerciseAnalyzer:
    def __init__(self):
        self.mp_drawing = mp.solutions.drawing_utils
        self.mp_pose = mp.solutions.pose
        self.data: List[List[float]] = []
        self.current_exercise: Optional[str] = None
        self.recording: bool = False
        self.video_writer: Optional[cv2.VideoWriter] = None
        self.counter: dict = {}
        self.draw_pose: bool = False
        self.start_detected: bool = False
        self.end_detected: bool = False
        self.previous_knee_angle: Optional[float] = None
        self.previous_back_angle: Optional[float] = None
        self.failed_lift: bool = False
        self.frame_dir: Optional[str] = None

    def calculate_angle(self, a: np.ndarray, b: np.ndarray, c: np.ndarray) -> float:
        """Calculate the angle between three points."""
        radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
        angle = np.abs(radians * 180.0 / np.pi)
        return 360 - angle if angle > 180.0 else angle

    def determine_label(self, angle: float, exercise: str) -> int:
        """Determine if the lift is successful based on the angle and exercise type."""
        if exercise == "deadlift":
            return 1 if 160 <= angle <= 180 else 0
        elif exercise == "squat":
            return 1 if angle <= 90 else 0
        elif exercise == "bench_press":
            return 1 if 70 <= angle <= 110 else 0
        return 0

    def analyze_deadlift(self, landmarks: List[any]) -> Tuple[float, float, int, bool, bool]:
        """Analyze the deadlift exercise."""
        shoulder = [landmarks[self.mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                    landmarks[self.mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
        hip = [landmarks[self.mp_pose.PoseLandmark.LEFT_HIP.value].x,
               landmarks[self.mp_pose.PoseLandmark.LEFT_HIP.value].y]
        knee = [landmarks[self.mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                landmarks[self.mp_pose.PoseLandmark.LEFT_KNEE.value].y]
        ankle = [landmarks[self.mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
                 landmarks[self.mp_pose.PoseLandmark.LEFT_ANKLE.value].y]

        back_angle = self.calculate_angle(shoulder, hip, knee)
        knee_angle = self.calculate_angle(hip, knee, ankle)

        if knee_angle < 45 and back_angle < 45 and not self.start_detected:
            self.start_detected = True
            self.end_detected = False
            self.failed_lift = False

        if knee_angle > 160 and back_angle > 160 and self.start_detected:
            self.end_detected = True

        if self.start_detected and not self.end_detected:
            if self.previous_knee_angle is not None and self.previous_back_angle is not None:
                if knee_angle < self.previous_knee_angle - 20 or back_angle < self.previous_back_angle - 20:
                    self.failed_lift = True

        self.previous_knee_angle = knee_angle
        self.previous_back_angle = back_angle

        label = 1 if self.start_detected and self.end_detected and not self.failed_lift else 0

        return back_angle, knee_angle, label, self.start_detected, self.end_detected

    def analyze_squat(self, landmarks: List[any]) -> Tuple[float, float, int, bool, bool]:
        """Analyze the squat exercise."""
        hip = [landmarks[self.mp_pose.PoseLandmark.LEFT_HIP.value].x,
               landmarks[self.mp_pose.PoseLandmark.LEFT_HIP.value].y]
        knee = [landmarks[self.mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                landmarks[self.mp_pose.PoseLandmark.LEFT_KNEE.value].y]
        ankle = [landmarks[self.mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
                 landmarks[self.mp_pose.PoseLandmark.LEFT_ANKLE.value].y]

        knee_angle = self.calculate_angle(hip, knee, ankle)
        hip_angle = self.calculate_angle([landmarks[self.mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, 
                                          landmarks[self.mp_pose.PoseLandmark.LEFT_SHOULDER.value].y], 
                                         hip, knee)

        if knee_angle > 160 and hip_angle > 160 and not self.start_detected:
            self.start_detected = True
            self.end_detected = False

        if knee_angle < 90 and hip_angle < 90 and self.start_detected:
            self.end_detected = True

        label = 1 if self.start_detected and self.end_detected else 0

        return knee_angle, hip_angle, label, self.start_detected, self.end_detected

    def analyze_bench_press(self, landmarks: List[any]) -> Tuple[float, int, bool, bool]:
        """Analyze the bench press exercise."""
        shoulder = [landmarks[self.mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                    landmarks[self.mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
        elbow = [landmarks[self.mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                 landmarks[self.mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
        wrist = [landmarks[self.mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                 landmarks[self.mp_pose.PoseLandmark.LEFT_WRIST.value].y]

        elbow_angle = self.calculate_angle(shoulder, elbow, wrist)

        if elbow_angle > 160 and not self.start_detected:
            self.start_detected = True
            self.end_detected = False

        if elbow_angle < 70 and self.start_detected:
            self.end_detected = True

        label = 1 if self.start_detected and self.end_detected else 0

        return elbow_angle, label, self.start_detected, self.end_detected

    def process_frame(self, image: np.ndarray, pose: mp.solutions.pose.Pose, exercise_type: Optional[str] = None, collect_data: bool = False) -> np.ndarray:
        """Process a single frame with MediaPipe and overlay results."""
        try:
            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image_rgb.flags.writeable = False
            results = pose.process(image_rgb)
            image_rgb.flags.writeable = True
            image_bgr = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR)

            if results.pose_landmarks:
                landmarks = results.pose_landmarks.landmark

                if exercise_type == "deadlift":
                    back_angle, knee_angle, label, self.start_detected, self.end_detected = self.analyze_deadlift(landmarks)
                    self.overlay_text(image_bgr, [
                        f'Back Angle: {back_angle:.2f}',
                        f'Knee Angle: {knee_angle:.2f}',
                        f'Label: {label}'
                    ])
                    if collect_data:
                        self.save_features(landmarks, exercise_type)

                elif exercise_type == "squat":
                    knee_angle, hip_angle, label, self.start_detected, self.end_detected = self.analyze_squat(landmarks)
                    self.overlay_text(image_bgr, [
                        f'Knee Angle: {knee_angle:.2f}',
                        f'Hip Angle: {hip_angle:.2f}',
                        f'Label: {label}'
                    ])
                    if collect_data:
                        self.save_features(landmarks, exercise_type)

                elif exercise_type == "bench_press":
                    elbow_angle, label, self.start_detected, self.end_detected = self.analyze_bench_press(landmarks)
                    self.overlay_text(image_bgr, [
                        f'Elbow Angle: {elbow_angle:.2f}',
                        f'Label: {label}'
                    ])
                    if collect_data:
                        self.save_features(landmarks, exercise_type)

                if self.draw_pose:
                    self.mp_drawing.draw_landmarks(image_bgr, results.pose_landmarks, self.mp_pose.POSE_CONNECTIONS)

            return image_bgr
        except Exception as e:
            print(f"Error processing frame: {e}")
            return image

    def overlay_text(self, image: np.ndarray, text_list: List[str]) -> None:
        """Overlay text on the image."""
        for i, text in enumerate(text_list):
            cv2.putText(image, text, (50, 50 + i*50), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

    def save_features(self, landmarks: List[any], exercise: str) -> None:
        """Save exercise features for later analysis."""
        if exercise == "deadlift":
            back_angle, knee_angle, label, _, _ = self.analyze_deadlift(landmarks)
            self.data.append([back_angle, knee_angle, label])
        elif exercise == "squat":
            knee_angle, hip_angle, label, _, _ = self.analyze_squat(landmarks)
            self.data.append([knee_angle, hip_angle, label])
        elif exercise == "bench_press":
            elbow_angle, label, _, _ = self.analyze_bench_press(landmarks)
            self.data.append([elbow_angle, label])

    def start_live_feed(self, capture: cv2.VideoCapture, window: tk.Tk, canvas: tk.Canvas, pose: mp.solutions.pose.Pose) -> None:
        """Start the live video feed."""
        try:
            ret, frame = capture.read()
            if ret:
                frame = self.process_frame(frame, pose, self.current_exercise)

                if self.recording and self.video_writer:
                    self.video_writer.write(frame)
                    frame_filename = f"{self.frame_dir}/frame_{int(capture.get(cv2.CAP_PROP_POS_FRAMES))}.jpg"
                    cv2.imwrite(frame_filename, frame)

                image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGBA)
                captured_image = Image.fromarray(image)
                photo_image = ImageTk.PhotoImage(image=captured_image)

                canvas.create_image(0, 0, anchor=tk.NW, image=photo_image)
                canvas.photo_image = photo_image

            window.after(10, self.start_live_feed, capture, window, canvas, pose)
        except Exception as e:
            print(f"Error in live feed: {e}")

    def setup_exercise(self, exercise: str) -> None:
        """Set the current exercise type."""
        self.current_exercise = exercise

    def toggle_draw_pose(self) -> None:
        """Toggle drawing of pose connections."""
        self.draw_pose = not self.draw_pose

    def generate_filename(self, exercise: str) -> str:
        """Generate a unique filename for the recording."""
        if exercise not in self.counter:
            self.counter[exercise] = 0
        self.counter[exercise] += 1
        timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        base_directory = f"{exercise}"
        video_directory = f"{base_directory}/video"
        frame_directory = f"{base_directory}/photos"

        if not os.path.exists(video_directory):
            os.makedirs(video_directory)
        if not os.path.exists(frame_directory):
            os.makedirs(frame_directory)

        self.frame_dir = f"{frame_directory}/frames_{timestamp}_{self.counter[exercise]}"
        if not os.path.exists(self.frame_dir):
            os.makedirs(self.frame_dir)

        return f"{video_directory}/{exercise}_{timestamp}_{self.counter[exercise]}.avi"

    def start_recording(self) -> None:
        """Start recording the exercise."""
        if self.current_exercise:
            filename = self.generate_filename(self.current_exercise)
            self.recording = True
            self.video_writer = cv2.VideoWriter(
                filename, cv2.VideoWriter_fourcc(*"MJPG"), 10, (CONFIG['FRAME_WIDTH'], CONFIG['FRAME_HEIGHT'])
            )
            print(f"Recording started: {filename}")

    def stop_recording(self) -> None:
        """Stop recording the exercise."""
        self.recording = False
        if self.video_writer:
            self.video_writer.release()
            self.video_writer = None
            print("Recording stopped")


def setup_ui(window: tk.Tk, analyzer: ExerciseAnalyzer) -> Tuple[tk.Canvas, tk.Frame]:
    """Set up the user interface."""
    canvas = tk.Canvas(
        window,
        width=CONFIG["FRAME_WIDTH"],
        height=CONFIG["FRAME_HEIGHT"] - 200,
        highlightthickness=0,
        bd=0,
    )
    canvas.pack()

    button_frame = tk.Frame(window)
    button_frame.configure(bg=CONFIG["DARK_MODE_BG_COLOR"])
    button_frame.pack(pady=20)

    deadlift_button = ck.CTkButton(
        button_frame,
        text="Deadlift",
        command=lambda: analyzer.setup_exercise("deadlift"),
        height=40,
        width=120,
        font=("Arial", 20),
        text_color=CONFIG["BUTTON_TEXT_COLOR"],
        fg_color=CONFIG["BUTTON_BG_COLOR"],
        hover_color=CONFIG["BUTTON_HOVER_COLOR"],
    )
    deadlift_button.grid(row=0, column=0, padx=10, pady=10)

    squat_button = ck.CTkButton(
        button_frame,
        text="Squat",
        command=lambda: analyzer.setup_exercise("squat"),
        height=40,
        width=120,
        font=("Arial", 20),
        text_color=CONFIG["BUTTON_TEXT_COLOR"],
        fg_color=CONFIG["BUTTON_BG_COLOR"],
        hover_color=CONFIG["BUTTON_HOVER_COLOR"],
    )
    squat_button.grid(row=0, column=1, padx=10, pady=10)

    bench_button = ck.CTkButton(
        button_frame,
        text="Bench Press",
        command=lambda: analyzer.setup_exercise("bench_press"),
        height=40,
        width=120,
        font=("Arial", 20),
        text_color=CONFIG["BUTTON_TEXT_COLOR"],
        fg_color=CONFIG["BUTTON_BG_COLOR"],
        hover_color=CONFIG["BUTTON_HOVER_COLOR"],
    )
    bench_button.grid(row=0, column=2, padx=10, pady=10)

    draw_button = ck.CTkButton(
        button_frame,
        text="Draw",
        command=analyzer.toggle_draw_pose,
        height=40,
        width=120,
        font=("Arial", 20),
        text_color=CONFIG["BUTTON_TEXT_COLOR"],
        fg_color=CONFIG["DRAW_BUTTON_COLOR"],
        hover_color=CONFIG["DRAW_BUTTON_HOVER_COLOR"],
    )
    draw_button.grid(row=1, column=0, padx=10, pady=20)

    record_button = ck.CTkButton(
        button_frame,
        text="Start Recording",
        command=analyzer.start_recording,
        height=40,
        width=150,
        font=("Arial", 20),
        text_color=CONFIG["BUTTON_TEXT_COLOR"],
        fg_color=CONFIG["BUTTON_START_COLOR"],
        hover_color=CONFIG["BUTTON_START_HOVER_COLOR"],
    )
    record_button.grid(row=1, column=1, padx=10, pady=20)

    stop_button = ck.CTkButton(
        button_frame,
        text="Stop Recording",
        command=analyzer.stop_recording,
        height=40,
        width=120,
        font=("Arial", 20),
        text_color=CONFIG["BUTTON_TEXT_COLOR"],
        fg_color=CONFIG["BUTTON_STOP_COLOR"],
        hover_color=CONFIG["BUTTON_STOP_HOVER_COLOR"],
    )
    stop_button.grid(row=1, column=2, padx=10, pady=20)

    return canvas, button_frame


def main():
    analyzer = ExerciseAnalyzer()
    capture = cv2.VideoCapture(CONFIG["CAPTURE_DEVICE_INDEX"])
    capture.set(3, CONFIG["FRAME_WIDTH"])
    capture.set(4, CONFIG["FRAME_HEIGHT"])

    window = tk.Tk()
    window.geometry(f"{CONFIG['FRAME_WIDTH']}x{CONFIG['FRAME_HEIGHT']}")
    window.title("Exercise Analyzer")
    window.configure(bg=CONFIG["DARK_MODE_BG_COLOR"])
    ck.set_appearance_mode("dark")

    canvas, _ = setup_ui(window, analyzer)

    with mp.solutions.pose.Pose(
        min_detection_confidence=0.5, min_tracking_confidence=0.5
    ) as pose:
        analyzer.start_live_feed(capture, window, canvas, pose)
        window.mainloop()

    capture.release()
    cv2.destroyAllWindows()


if __name__ == "__main__":
    main()